# Procesamiento de datos Encuesta Nacional de Hogares (INE)

### Fundamientación y objetivo

El INE libera sus datos en diversos formatos, comunmente con extension .sav en conjunto con un diccionario de datos de extensión .xlsx.

La forma que está construido este diccionario dificulta decodificar los datos de la encuesta, tanto a nivel de los nombres de las diferentes columnas como de los valores de los códigos de cada variable. 

Se busca entonces en esta variable concatenar las encuestas del 2020 y 2021 (cuyo dicciorio de variables es identico) en un mismo dataset y decodificarlo. 

Sin embargo, la gran cantidad de columnas de diferente origen dificultan el análisis comprensivo, por lo que al final de la libreta se separan las variables en diferentes categorías y se genera una función para generar datasets personalizados según las categorias que se deseen. 

Los datos serán luego analizados en diferentes libretas personales así como para su análisis en Complexus Analytics. 

In [66]:
# Primero los imports necesarios. 
from glob import glob
import pandas as pd
import numpy as np
from pyreadstat import read_sav

### Porque es necesaria esta libreta

La decodificación puede hacerse programaticamente si tenemos un buen diccionario; simplemente necesitamos tener apareados códigos y descripciones de códigos en relación 1:1 para hacer el mapeo.  

Sin embargo, una inspeccion de la encuesta muestra que hay varias variables con el mismo codigo. 

In [83]:
df_dict_persona = pd.read_excel('DICCIONARIO ECH 2021_semestre1_terceros.xlsx',header=4,sheet_name='PERSONA')
columns_rename2 = {df_dict_persona.columns[0]:'Variable',
                    df_dict_persona.columns[1]:'Codigo'}
df_dict_persona.rename(columns=columns_rename2,inplace=True)
df_dict_persona = df_dict_persona[['Variable','Codigo']].dropna(axis=0)
df_dict_persona.tail(100) ###

,Variable,Codigo
641,"ALIMENTOS, BEBIDAS O ALGÚN OTRO TIPO DE RETRIB...",g_id_3a
645,SUELDO O JORNALES LÍQUIDOS,g126_1
646,"COMISIONES, INCENTIVOS, HORAS EXTRAS, HABILITA...",g126_2
647,VIÁTICOS NO SUJETOS A RENDICIÓN,g126_3
648,PROPINAS,g126_4
...,...,...
821,TOTAL DE INGRESOS PERSONALES,PT1
822,INGRESOS DE LA OCUPACIÓN PRINCIPAL,PT2
823,TOTAL DE INGRESOS POR TRABAJO,PT4
824,POBREZA SEGÚN METODOLOGÍA 2006,pobre06


Además, hay descripciones de variables que están repetidas 

In [88]:
print([df_dict_persona['Variable'].value_counts()>1]) 

[TIPO DE CENTRO EDUCATIVO                                             True
FINALIZÓ EL NIVEL                                                    True
CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ                                True
RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO                           True
CANTIDAD DE COMIDAS SEMANALES                                        True
                                                                    ...  
INGRESO POR MEDIANERÍA O APARCERÍA                                  False
COLABORÓ EN NEGOCIO FAMILIAR O TRABAJO CONCRETO LA SEMANA PASADA    False
MECANISMO ACCESO IAMC                                               False
NUEVO MIEMBRO DEL HOGAR                                             False
A QUÉ SE DEDICA LA EMPRESA QUE LO CONTRATA                          False
Name: Variable, Length: 222, dtype: bool]


'TIPO DE CENTRO EDUCATIVO' esta 13 veces en el diccionario, lo que hace necesaria una inspeccion visual de los códigos. 

In [ ]:
df_dict_persona[df_dict_persona['Variable']=='TIPO DE CENTRO EDUCATIVO'].shape[0]

#### Comienza el procesamiento

Dentro de la carpeta "Data" se encuentran archivos de datos de la ENH; primero hay que descomprimirlos

In [112]:
import zipfile
with zipfile.ZipFile("data/data.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")

In [113]:
all_files = glob("data/*.sav")
df = pd.DataFrame()
for file in all_files:
    df_sav  ,metadata = read_sav(file, encoding="latin1")
    upp_cols = dict(zip(df_sav.columns,[x.upper() for x in df_sav.columns])) ### Paso todos los nombres de columnas a mayusculas
    df_sav.rename(columns=upp_cols,inplace=True)
    df = pd.concat([df,df_sav])

##### Vemos que el dataset tiene una gran cantidad de columnas (404 correspondiente a las diferentes preguntas que se hacen en la cuesta). 

Esta es la razón para dividir el dataset más adelante

In [89]:
print(f"El dataste tiene {df.shape[0]} columnas y {df.shape[1]} columnas")

El dataste tiene 209637 columnas y 404 columnas


In [90]:
columns = list(df.columns)
duplicated = [column for column in df.columns if columns.count(column) > 1]
duplicated

['OTRA',
 'TIPO DE CENTRO EDUCATIVO',
 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'TIPO DE CENTRO EDUCATIVO',
 'CANTIDAD DE DÍAS QUE ASISTE A LA SEMANA',
 'CANTIDAD DE HORAS AL DÍA',
 'FALTÓ ALGÚN DÍA',
 'RAZONES POR LAS QUE NO ASISTIÓ',
 'TIPO DE CENTRO EDUCATIVO',
 'CANTIDAD DE DÍAS QUE ASISTE A LA SEMANA',
 'CANTIDAD DE HORAS AL DÍA',
 'FALTÓ ALGÚN DÍA',
 'RAZONES POR LAS QUE NO ASISTIÓ',
 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'FINALIZÓ EL NIVEL',
 'TIPO DE CENTRO EDUCATIVO',
 'TIPO DE CENTRO EDUCATIVO',
 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'FINALIZÓ EL NIVEL',
 'TIPO DE CENTRO EDUCATIVO',
 'TIPO DE CENTRO EDUCATIVO',
 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'TIPO DE CENTRO EDUCATIVO',
 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'FINALIZÓ EL NIVEL',
 'TIPO DE CENTRO EDUCATIVO',
 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'FINALIZÓ EL NIVEL',
 'TIPO DE CENTRO EDUCATIVO',
 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'FINALIZÓ EL NIVEL',
 'TIPO DE CENTRO EDUC

#### Codificando departamentos

In [91]:
deps = ['Montevideo',
 'Artigas',
 'Canelones',
 'Cerro Largo',
 'Colonia',
 'Durazno',
 'Flores',
 'Florida',
 'Lavalleja',
 'Maldonado',
 'Paysandu',
 'Rio Negro',
 'Rivera',
 'Rocha',
 'Salto',
 'San José',
 'Soriano',
 'Tacuarembó',
 'Treinta y Tres']
mydeps = { k:v for (k,v) in zip(range(1,20), deps)}  
mydeps

{1: 'Montevideo',
 2: 'Artigas',
 3: 'Canelones',
 4: 'Cerro Largo',
 5: 'Colonia',
 6: 'Durazno',
 7: 'Flores',
 8: 'Florida',
 9: 'Lavalleja',
 10: 'Maldonado',
 11: 'Paysandu',
 12: 'Rio Negro',
 13: 'Rivera',
 14: 'Rocha',
 15: 'Salto',
 16: 'San José',
 17: 'Soriano',
 18: 'Tacuarembó',
 19: 'Treinta y Tres'}

In [92]:
# Hogar

region_3 = {1:'Montevideo',
            2: 'Loc. urbana de mas de 5000 habitantes',
            3: 'Loc. urbana de menos de 5000 habitantes y zona rural'}

region_4 = {1:'Montevideo',
            2: 'Loc. urbana de mas de 5000 habitantes',
            3: 'Loc. urbana de menos de 5000 habitantes',
            4: 'Zona rural'}

H5_CV = {1:'Propietario/a de la vivienda y el terreno y los está pagando',
                     2: 'Propietario/a de la vivienda y el terreno y ya los pagó',
                     3: 'Propietario/a solamente de la vivienda y la está pagando',
                     4: 'Propietario/a solamente de la vivienda y ya la pagó',
                     5: 'Inquilino/a o arrendatario/a de la vivienda',
                     6: 'Ocupante con relación de dependencia',
                     7: 'Ocupante gratuito y se lo permite el B.P.S.',
                     8: 'Ocupante gratuito y se lo permite un particular',
                     9: 'Ocupante sin permiso del propietario/a',
                     10: 'Miembro de cooperativa de vivienda'
                    }

si_no = {1:'Si',
       2:'No',
        99:'Sin dato'}

h167_1 = {1: 'Si y genera intereses',
         3: 'Si y No genera intereses',
         2:'No'}

pobre06 = {0:'No pobre',
          1: 'Pobre'}

indigente06 = {0:'No indigente',
              1:'Indigente'}

In [93]:
#Persona

ESTRED13 = {1:'Montevideo - Nivel económico bajo',
            2:'Montevideo - Nivel económico medio - bajo',
            3:'Montevideo - Nivel económico medio',
            4:'Montevideo - Nivel económico medio - alto',
            5:'Montevideo - Nivel económico alto',
            6:'Zona metropolitana',
            7:'Interior Norte (Artigas, Rivera, Cerro Largo, Treinta y Tres)',
            8:'Costa Este (Canelones, Maldonado, Rocha)',
            9:'Litoral Norte (Salto, Paysandú, Río Negro)',
            10: 'Litoral Sur (Soriano, Colonia, San José)',
            11:'Centro Norte (Tacuarembó, Durazno)',
            12:'Centro Sur (Flores, Florida, Lavalleja)'
           }

e557 ={1:'Sí, es la misma persona',
      2:'No, responde un miembro de este hogar',
      3:'No, responde un miembro calificado de otro hogar'}

e26 = {1: ' Hombre',
      2: 'Mujer'}

e30 = {1:'Jefe/a',
      2:'Esposo/a o compañero/a',
      3:'Hijo/a de ambos',
      4:'Hijo/a sólo del jefe',
      5:'Hijo/a sólo del esposo/a compañero/a',
      6:'Yerno/nuera',
      7:'Padre/madre',
      8:'Suegro/a',
      9:'Hermano/a',
      10:'Cuñado/a',
      11:'Nieto/a',
      12:'Otro pariente',
      13:'Otro no pariente',
      14:'Servicio doméstico o familiar del mismo'}

publico_privado = {1:'Público',
2:'Privado',
99:'Sin dato'
}

E45_CV = {1:'MSP/ASSE',
        2:'Mutualista (IAMC)',
        3:'Seguro Médico Privado',
        4:'Hospital Policial / Militar',
        5:'Área de salud del BPS',
        6:'Policlínica municipal',
        7:'No cuenta con derechos en ninguna institución'}

E45_1_1_CV = { 1:'A través de Fonasa, miembro de este hogar',
4:'A través de Fonasa, miembro de otro hogar',
2:'Por bajos recursos',
3:'Pagando arancel',
5:'A través de un miembro de este hogar (sin ser FONASA)',
6:'A través de otro que no es miembro del hogar (sin ser FONASA)'}
    
E45_2_1_CV = { 1:'A través de Fonasa, miembro de este hogar',
6:'A través de Fonasa, miembro de otro hogar',
2:'Paga un miembro de este hogar',
5:'Paga el empleador de un miembro del hogar',
3:'A través de otro que no es miembro del hogar (sin ser FONASA incluye exempleador)'
}

E45_3_1_CV = {1:'A través de Fonasa, miembro de este hogar pagando complemento',
6:'A través de Fonasa, miembro de otro hogar pagando complemento',
2:'Paga un miembro de este hogar',
5:'Paga el empleador de un miembro del hogar',
3:'A través de otro que no es miembro del hogar (sin ser Fonasa, incluye exempleador)'}

E45_4_1_CV = {1:'A través de un miembro de este hogar',
2:'A través de un miembro de otro hogar'
}

E47_CV = {1:'Está incluida en la cuota de la IAMC',
2:'Paga un miembro de este hogar',
3:'Paga un miembro de otro hogar',
4:'Paga el empleador de un miembro del hogar'
}

e50_CV = {1:'Público',
2:'Privado'}

e562_CV = {1:'Sí, beca MEC',
2:'Sí, compromiso educativo',
3:'No recibe'
}

e239 = {1:'Público',
2:'Privado',
3:'CAIF/CAPI/Nuestros niños',
99:'Sin dato',
}

e242 = {1:'Por vacaciones',
2:'Porque faltó el/la maestro/a',
3:'Por mal tiempo',
4:'Por enfermedad',
6:'Por indisposicion de tercero o responsable',
5:'Otro',
99:'Sin dato',
}

e193 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato',
}

e194 = {1:'Público',
2:'Privado',
3:'CAIF/CAPI/Nuestros niños',
99:'Sin dato',
}

e245 = {1:'Por vacaciones',
2:'Porque faltó el/la maestro/a',
3:'Por mal tiempo',
4:'Por enfermedad',
6:'Por indisposición de tercero o responsable',
5:'Otro',
}

e197 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato',
}

e198 = {1:'Público',
2:'Privado',
99:'Sin dato'}

e199 = {1:'Común',
2:'Tiempo completo',
99:'Sin dato'
}


e210_1 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e210_2 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e210_3 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e213 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e216 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e212 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato'
}

e51_7_1 = {1:'Enseñanza secundaria completa',
2:'Ciclo Básico, liceo o UTU',
3:'Enseñanza primaria completa',
4:'Ninguna',
99:'Sin dato',
          }
e215 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato'
}

e218 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato',
}

e219 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e221 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato',
}

e222 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e224 = {1:'Asiste actualmente',
2:'Sí, asistió',
3:'No asistió',
99:'Sin dato',
}

e225 = {1:'Público',
2:'Privado',
99:'Sin dato'
}

e246 = {7:'Celíaco',
14:'Canasta MIDES ',
11:'Otra'
}

f69 = {1:'Por licencia (vacaciones, enfermedad, estudio, cuidados, etc.)',
2:'Por poco trabajo o mal tiempo',
3:'Por estar en seguro de paro o seguro de desempleo',
5:'Por atender hijos/as enfermos/as o cuidar hijos/as',
6:'Por atender otras personas dependientes del hogar',
7:'Por huelga o conflicto laboral',
8:'Por estar suspendido/a',
9:'Porque me despidieron',
10:'Suspendido por el COVID-19',
11:'Por estar en cuarentena por COVID-19',
14:'Por estar en seguro de paro especial por COVID-19'
}

sabe_nosabe = {1:'Si',
2:'No',
3:'No sabe'
}

f275 = {1:'Dentro de 4 semanas o menos',
2:'Entre 5 y 12 semanas',
3:'En más de 12 semanas',
4:'No sabe cuándo volverá a trabajar'
}

f275_1 = {1:'Si',
2:'No',
3:'No sabe'
}

f277 = {1:'Sí, más de la mitad de lo que produce',
2:'Sí, menos de la mitad de lo que produce',
3:'No, nada'
}


f73 = {1:'Asalariado/a privado/a',
2:'Asalariado/a público/a',
3:'Miembro de cooperativa de producción o trabajo',
4:'Patrón/a',
5:'Cuenta propia sin local ni inversión',
6:'Cuenta propia con local o inversión',
7:'Miembro del hogar no remunerado',
8:'Trabajador/a de un programa social de empleo'
}

f74 = {1:'Administración Central',
3:'Entes Autónomos y Servicios Descentralizados',
2:'Otros Organismos del 220',
4:'Gobiernos departamentales',
5:'Otros',
}

f264 = { 1:'Empresa unipersonal',
2:'Sociedad de hecho / Sociedad civil',
3:'Sociedad anónima (S.A.)',
4:'Sociedad de responsabilidad limitada (S.R.L.) u otras sociedades personales con contrato',
5:'Cooperativa',
6:'Otras',
7:'No tiene'}

f265 = {1:'Monotributo',
2:'Monotributo social (MIDES)',
3:'Pequeña empresa (IVA mínimo)',
4:'Servicios personales (IVA, IRPF, y/o IRAE)',
5:'Régimen general (IVA, IRAE, e IP)',
6:'Otra forma de tributación',
7:'No tributa'
}

f77 = {1:'Una persona',
2:'2 a 4 personas',
3:'5 a 9 personas',
6:'10 a 19 personas',
7:'20 a 49 personas',
5:'50 o más personas'
}
f78 = {1:'En un establecimiento fijo',
2:'En su vivienda,',
3:'A domicilio',
4:'En la calle, en un puesto o lugar fijo',
5:'En la calle, en un puesto móvil',
6:'En la calle, desplazándose',
7:'En la vía pública',
8:'En un predio agropecuario o marítimo'
}

f80 = {1:'Sí',
2:'No',
3:'En otro país'
}

f83 = {1:'BPS',
2:'BPS y AFAP',
3:'Policial',
4:'Militar',
5:'Profesional',
6:'Notarial',
7:'Bancaria',
8:'En el exterior',
}

f266 = {1:'Si',
2:'No',
3:'La empresa / institución no permite realizar horas extras'}

f266_1 = {1:'Mediante pago',
2:'Mediante derecho a compensarlas',
3:'Mediante días adicionales de vacaciones'
}

f266_2 = {1:'Cuenta con horario flexible',
2:'No se las reconocen'
}


f101 = {1:'Trabajar más horas',
2:'Trabajar menos horas',
3:'Trabajar menos horas para cuidar a sus hijos/as',
4:'Trabajar menos horas para cuidar personas dependientes del hogar',
5:'Un empleo más adecuado a su formación',
6:'El trabajo actual no es estable',
7:'Para mejorar las condiciones del trabajo actual (cercanía, clima laboral, etc.)',
8:'Mayor ingreso',
9:'Otros'
}

f104 = {1:'Por razones de estudio',
2:'Por razones de salud',
3:'Por tener que cuidar niños/as',
4:'Por tener que cuidar personas dependientes del hogar',
5:'Porque no consigue más trabajo',
6:'Por razones personales',
}

f105 = {1:'Puso o contestó avisos en diarios, etc.',
2:'Consultó con agencias de empleo',
3:'Consultó directamente con el empleador',
4:'Consultó con amigos o parientes',
5:'Hizo trámites, gestionó préstamos, buscó local o campo para establecerse',
6:'Consultó por Internet',
7:'Nada',
}

f106 ={ 1:'Sí, ahora mismo',
2:'Sí, en otra época del año',
3:'No',
99:'Sin dato',
}

f108 = {1:'Está incapacitado físicamente',
2:'Tiene trabajo que comenzará en los próximos 30 días',
3:'Está esperando el resultado de gestiones ya emprendidas',
4:'Buscó antes, no encontró y dejó de buscar',
5:'No tiene tiempo por su trabajo doméstico, el cuidado de niños/as o personas dependientes',
7:'Por motivos de salud temporales',
8:'Por motivos de estudio',
9:'Está a la espera de una estación de mayor actividad',
10:'Es jubilado, pensionista o muy mayor de edad',
11:'No quiere o no necesita trabajar',
12:'Otras causas',
13:'No buscó trabajo por la coyuntura de la pandemia'
}

f110 = {1:'Puso o contestó avisos en diarios, etc.',
2:'Consultó con agencias de empleo',
3:'Consultó directamente con el empleador',
4:'Consultó con amigos o parientes',
5:'Hizo trámites, gestionó préstamos, buscó local o campo para establecerse',
6:'Consultó por Internet',
7:'Nada'}

f112 = {1:'Jornada de baja carga horaria',
2:'Horario especial',
3:'Acorde a su conocimiento o experiencia',
4:'Condiciones salariales',
5:'Condición del lugar de trabajo o personales',
6:'Horario flexible para atender las necesidades familiares'
}


f122 = {1:'Despido',
2:'Cierre deL establecimiento',
3:'Finalización del contrato',
4:'Acabó la zafra',
5:'Mal pago',
6:'Razones de estudio',
7:'Razones familiares',
8:'Se jubiló',
9:'Renuncia por otras razones',
99:'Sin dato'
}

f125 = {1:'Pensión por vejez',
2:'Pensión por fallecimiento',
3:'Pensión por invalidez',
4:'Pensión del extranjero',
5:'Pensión a las víctimas de delitos violentos',
6:'Pensión para hijos de fallecidos por violencia doméstica',
7:'Pensión Especial Reparatoria',
8:'Pensión Reparatoria Ley Integral para Personas Trans',
}

g140 = {1:'Sí',
2:'No',
3:'No corresponde'
}

pobpcoac = {1:'Menores de 14 años',
2:'Ocupados',
3:'Desocupados buscan trabajo por primera vez',
4:'Desocupados propiamente dichos',
5:'Desocupados en seguro de paro',
6:'Inactivo, realiza quehaceres del hogar',
7:'Inactivo, estudiante',
8:'Inactivo, rentista',
9:'Inactivo, pensionista',
10:'Inactivo, jubilado',
11:'Inactivo, otro'
           }
TIPO_OCUPADO = {1:'Ocupado trabajando',
2:'Ocupado ausente'}

f268 = {1:'Si',
2:'No',
3:'No sabe'
}

f100 ={ 1:'Sustituir',
2:'Complementar'}


g256 = {1:'Incluidas en el sueldo',
2:'Por separado del sueldo en un local de cobro',
}

g152 = {1:'Todos los meses',
2:'Cada dos meses'
}

pobpcoac = {
1:'Menores de 14 años',
2:'Ocupados',
3:'Desocupados buscan trabajo por primera vez',
4:'Desocupados propiamente dichos',
5:'Desocupados en seguro de paro',
6:'Inactivo, realiza quehaceres del hogar',
7:'Inactivo, estudiante',
8:'Inactivo, rentista',
9:'Inactivo, pensionista',
10:'Inactivo, jubilado',
11:'Inactivo, otro'
}

ESTRED13 = {1:'ontevideo - Nivel económico bajo',
2:'Montevideo - Nivel económico medio - bajo',
3:'Montevideo - Nivel económico medio',
4:'Montevideo - Nivel económico medio - alto',
5:'Montevideo - Nivel económico alto',
6:'Zona metropolitana',
7:'Interior Norte (Artigas, Rivera, Cerro Largo, Treinta y Tres)',
8:'Costa Este (Canelones, Maldonado, Rocha)',
9:'Litoral Norte (Salto, Paysandú, Río Negro)',
10:'Litoral Sur (Soriano, Colonia, San José)',
11:'Centro Norte (Tacuarembó, Durazno)',
12:'Centro Sur (Flores, Florida, Lavalleja)'
}



ASCENDENCIA ={1:'Afro o negra',
2:'Asiática o amarilla',
3:'Blanca',
4:'Indígena',
5:'Otra',
99:'SIN DATO'}

E45_2_1_CV = {
        1:'A través de Fonasa, miembro de este hogar',
        6:'A través de Fonasa, miembro de otro hogar',
        2:'Paga un miembro de este hogar',
        5:'Paga el empleador de un miembro del hogar',
        3:'A través de otro que no es miembro del hogar (sin ser FONASA incluye exempleador)',
}

E45_CV = {1:'MSP/ASSE',
                2:'Mutualista (IAMC)',
                3:'Seguro Médico Privado',
                4:'Hospital Policial / Militar',
                5:'Área de salud del BPS',
                6:'Policlínica municipal',
                7:'No cuenta con derechos en ninguna institución'
}

E45_1_1_CV = {
                1:'A través de Fonasa, miembro de este hogar',
                4:'A través de Fonasa, miembro de otro hogar',
                2:'Por bajos recursos',
                3:'Pagando arancel',
                5:'A través de un miembro de este hogar (sin ser FONASA)',
                6:'A través de otro que no es miembro del hogar (sin ser FONASA)'
}


E45_3_1_CV = {  1:'A través de Fonasa, miembro de este hogar pagando complemento',
                6:'A través de Fonasa, miembro de otro hogar pagando complemento',
                2:'Paga un miembro de este hogar',
                5:'Paga el empleador de un miembro del hogar',
                3:'A través de otro que no es miembro del hogar (sin ser Fonasa, incluye exempleador)'
}

E45_4_1_CV = {1:'A través de un miembro de este hogar',
                2:'A través de un miembro de otro hogar'
}

In [94]:
df['DPTO'] = df['DPTO'].map(mydeps)
df['REGION_3'] = df['REGION_3'].map(region_3)
df['REGION_4'] = df['REGION_4'].map(region_4)
df['H5_CV'] = df['H5_CV'].map(region_4)
df['POBRE06'] = df['POBRE06'].map(si_no)
df['INDIGENTE06'] = df['INDIGENTE06'].map(si_no)
df['H155'] = df['H155'].map(si_no)
df['H156'] = df['H156'].map(si_no)
df['H161'] = df['H161'].map(si_no)
df['H159'] = df['H159'].map(si_no)
df['H162'] = df['H162'].map(si_no)
df['H227'] = df['H227'].map(si_no)
df['H269'] = df['H269'].map(si_no)
df['H167_2'] = df['H167_2'].map(si_no)
df['H167_3'] = df['H167_3'].map(si_no)
df['H167_4'] = df['H167_4'].map(si_no)
df['H169'] = df['H169'].map(si_no)
df['H271'] = df['H271'].map(si_no)
df['H171'] = df['H171'].map(si_no)
df['H172'] = df['H172'].map(si_no)
df['H173'] = df['H173'].map(si_no)
df['I228'] = df['I228'].map(si_no) #No esta en el df
df['I259'] = df['I259'].map(si_no)
df['H252'] = df['H252'].map(si_no)
df['H160'] = df['H160'].map(si_no)
df['E557'] = df['E557'].map(e557)
df['E26'] = df['E26'].map(e26)
df['E49'] = df['E49'].map(si_no)
df['E48'] = df['E48'].map(si_no)
df['E193'] = df['E193'].map(e193)
df['E30'] = df['E30'].map(e30)
df['E238'] = df['E238'].map(si_no)
df['E30'] = df['E30'].map(e30)
df['E241'] = df['E241'].map(si_no)
df['E242'] = df['E242'].map(e242)
df['E239'] = df['E239'].map(e194)
df['E194'] = df['E194'].map(e194)
df['E244'] = df['E244'].map(si_no)
df['E245'] = df['E245'].map(e242)
df['E196'] = df['E196'].map(si_no)
df['E197'] = df['E197'].map(e193)
df['E198'] = df['E198'].map(si_no)
df['E199'] = df['E199'].map(si_no)
df['E200'] = df['E200'].map(si_no)
df['E201'] = df['E201'].map(si_no)
df['E211'] = df['E211'].map(si_no)
df['E562'] = df['E562'].map(e562_CV)
df['E560'] = df['E560'].map(si_no)
df['E215'] = df['E215'].map(e193)
df['E218'] = df['E218'].map(e193)
df['E216'] = df['E216'].map(publico_privado)
df['E219'] = df['E219'].map(publico_privado)
df['E221'] = df['E221'].map(e193)
df['E222'] = df['E222'].map(publico_privado)
df['E224'] = df['E224'].map(e193)
df['E225'] = df['E225'].map(publico_privado)
df['E213'] = df['E213'].map(publico_privado)
df['E212'] = df['E212'].map(e193)
df['E559'] = df['E559'].map(si_no)
df['E561'] = df['E561'].map(si_no)
df['E59'] = df['E59'].map(si_no)
df['E246'] = df['E246'].map(e246)
df['F269'] = df['F269'].map(si_no)
df['F270'] = df['F270'].map(si_no)
df['F271'] = df['F271'].map(si_no)
df['F272'] = df['F272'].map(si_no)
df['F273'] = df['F273'].map(si_no)
df['F274'] = df['F274'].map(si_no)
df['F275'] = df['F275'].map(si_no)
df['F69'] = df['F69'].map(f69)
df['F276'] = df['F276'].map(si_no)
df['F277'] = df['F277'].map(f277)
df['F73'] = df['F73'].map(f73)
df['F74'] = df['F74'].map(f74)
df['F75'] = df['F75'].map(si_no)
df['F262'] = df['F262'].map(si_no)
df['F263'] = df['F263'].map(si_no)
df['F264'] = df['F264'].map(f264)
df['F265'] = df['F265'].map(f265)
df['F77'] = df['F77'].map(f77)
df['F78'] = df['F78'].map(f78)
df['F80'] = df['F80'].map(si_no)
df['F81'] = df['F81'].map(si_no)
df['F82'] = df['F82'].map(si_no)
df['F83'] = df['F83'].map(f83)
df['F84'] = df['F84'].map(si_no)
df['F266'] = df['F266'].map(f266)
df['F267'] = df['F267'].map(si_no)
df['F268'] = df['F268'].map(f268)
df['F86'] = df['F86'].map(si_no)
df['F89'] = df['F89'].map(si_no)
df['F92'] = df['F92'].map(f73)
df['F93'] = df['F93'].map(f77)
df['F94'] = df['F94'].map(f80)
df['F95'] = df['F95'].map(si_no)
df['F96'] = df['F96'].map(si_no)
df['F97'] = df['F97'].map(f83)
df['F98_2'] = df['F98_2'].map(si_no)
df['F98_3'] = df['F98_3'].map(si_no)
df['F99'] = df['F99'].map(si_no)
df['F100'] = df['F100'].map(f100)
df['F101'] = df['F101'].map(f101)
df['F102'] = df['F102'].map(si_no)
df['F103'] = df['F103'].map(si_no)
df['F104'] = df['F104'].map(f104)
df['F105'] = df['F105'].map(f105)
df['F106'] = df['F106'].map(f106)
df['F107'] = df['F107'].map(si_no)
df['F108'] = df['F108'].map(f108)
df['F109'] = df['F109'].map(si_no)
df['F110'] = df['F110'].map(f110)
df['F112'] = df['F112'].map(f112)
df['F111'] = df['F111'].map(si_no)
df['F114'] = df['F114'].map(si_no)
df['F115'] = df['F115'].map(si_no)
df['F116'] = df['F116'].map(si_no)
df['F117'] = df['F117'].map(si_no)
df['F123'] = df['F123'].map(si_no)
df['F121'] = df['F121'].map(f73)
df['F122'] = df['F122'].map(f122)
df['F124_1'] = df['F124_1'].map(si_no)
df['F124_2'] = df['F124_2'].map(si_no)
df['F124_3'] = df['F124_3'].map(si_no)
df['F124_4'] = df['F124_4'].map(si_no)
df['F124_5'] = df['F124_5'].map(si_no)
df['F125'] = df['F125'].map(f125)
df['G127'] = df['G127'].map(si_no)
df['G128'] = df['G128'].map(si_no)
df['G129'] = df['G129'].map(si_no)
df['G130'] = df['G130'].map(si_no)
df['G131'] = df['G131'].map(si_no)
df['G132'] = df['G132'].map(g140)
df['G133'] = df['G133'].map(si_no)
df['G135'] = df['G135'].map(si_no)
df['G136'] = df['G136'].map(si_no)
df['G137'] = df['G137'].map(si_no)
df['G138'] = df['G138'].map(si_no)
df['G139'] = df['G139'].map(si_no)
df['G140'] = df['G140'].map(si_no)
df['G141'] = df['G141'].map(si_no)
df['G144'] = df['G144'].map(si_no)
df['G149'] = df['G149'].map(si_no)
df['G150'] = df['G150'].map(si_no)
df['G255'] = df['G255'].map(si_no)
df['G256'] = df['G256'].map(g256)
df['G152'] = df['G152'].map(g152)
df['G153'] = df['G153'].map(si_no)
df['G154'] = df['G154'].map(si_no)
df['G258'] = df['G258'].map(si_no)
df['POBPCOAC'] = df['POBPCOAC'].map(pobpcoac)
df['ESTRED13'] = df['ESTRED13'].map(ESTRED13)
df['E29_1'] = df['E29_1'].map(si_no)
df['E29_4'] = df['E29_4'].map(si_no)
df['E29_5'] = df['E29_5'].map(si_no)
df['E29_3'] = df['E29_3'].map(si_no)
df['E29_2'] = df['E29_2'].map(si_no)
df['E29_6'] = df['E29_6'].map(ASCENDENCIA)
df['E45_CV'] = df['E45_CV'].map(E45_CV)
df['E45_1_1_CV'] = df['E45_1_1_CV'].map(E45_1_1_CV)
df['E45_2_1_CV'] = df['E45_2_1_CV'].map(E45_2_1_CV)
df['E45_3_1_CV'] = df['E45_3_1_CV'].map(E45_3_1_CV)
df['E45_4_1_CV'] = df['E45_4_1_CV'].map(E45_4_1_CV)

KeyError: 'DPTO'

In [ ]:
to_drop = (['NUMERO',
            "F108_1",#otra razon por la que no busco trabajo
            "F118_2",
            'BARRIO',  #Esta el nombre del barrio
           'PESOMEN',#No es necesario
           'H1_CV', #No especifica
           'H2_CV',  #No especifica,
            'MTO_CUOTA', #LOS SIGUIENTES "MTO" SON VALORES QUWE NO APORTAN AL ANALISIS
            'MTO_CABALL',
            'MTO_HOGCON',
            'MTO_DESAY', 
            'MTO_ALMUE', 
            'MTO_VACAS',
            'MTO_OVEJA',
            "F94_2",
            'INDACELIAC',
            'INDAEMER',
            "F264_1",
            "E242_1",
            "F264_1",
            "F80_2",
            "F88_1" ,#Anos que trabaja, tiene poco datos
            "HT11",
            "HT13",
            "H171_2",
            "E45_1_1_1_CV",
            "E45_2_1_1_CV",
            "E45_3_1_1_CV",
            "E45_4_1_1_CV",
            'LP_06',
            'LI_06',
            'E47_1_CV',
            'H1_CV',
            'H2_CV',
            'NPER',
            "F265_1",
            "G_ID_1",
            "G_ID_2",
            "G_ID_3",
            "G_ID_1A",
            "G_ID_2A",
            "G_ID_3A",
            "E245_1",
            "G154_1",
            "MTO_EMER",
            "H158_2",
            "H158_1",
            'LOC_AGR_13',
            "H166",
            'LECHEENPOL', #VALOR LECHE EN POLVO
            'E563', # SOlO EN 2020
            'E562_CV'
           ] 
          )
df.drop(to_drop,inplace=True,axis=1)

In [95]:
excluded = (['numero','anio','mes','SECC','SEGM','BARRIO','HOGAR','ccz','loc_agr_13','pesomen',
            'H1_CV','H2_CV',
            'loc_agr_13',
 'H1_CV',
 'h164',
 'h165',
 'H2_CV',
 'H155_1',
 'H156_1',
 'H252_1',
 'H158_1',
 'H158_2',
 'H160_1',
 'H160_2',
 'H163_1',
 'H163_2',
 'H269_1',
 'H167_1',
 'H167_1_1',
 'H167_1_2',
 'H167_2',
 'H167_2_1',
 'H167_2_2',
 'H167_3',
 'H167_3_1',
 'H167_3_2',
 'H167_4',
 'H167_4_1',
 'H167_4_2',
 'H170_1',
 'H170_2',
 'H271_1',
 'H171_1',
 'H171_2',
 'H172_1',
 'H173_1',
 'E0_CV',
 'E01_CV',
 'E1_CV',
 'e29_1',
 'e29_2',
 'e29_3',
 'e29_4',
 'e29_5',
 'e29_6',
 'E45_CV',
 'E45_1_1_CV',
 'E45_1_1_1_CV',
 'E45_2_1_CV',
 'E45_2_1_1_CV',
 'E45_3_1_CV',
 'E45_3_1_1_CV',
 'E45_4_1_CV',
 'E45_4_1_1_CV',
 'E46_CV',
 'E47_CV',
 'E47_1_CV',
 'E49_CV',
 'E50_CV',
 'E196_CV',
 'E196_1_CV',
 'E196_2_CV',
 'E196_3_CV',
 'e240_1',
 'e240_2',
 'e243_1',
 'e243_2',
 'e196_1',
 'e196_2',
 'e196_3',
 'e197_1',
 'e51_2',
 'e51_3',
 'e200_1',
 'e200_2',
 'e200_3',
 'e201_1',
 'e202_1',
 'e202_9',
 'e202_10',
 'e202_3',
 'e202_4',
 'e202_8',
 'e202_5',
 'e202_6',
 'e202_7',
 'e51_4',
 'e210_1',
 'e51_5',
 'e210_2',
 'e51_6',
 'e209_1',
 'e210_3',
 'e211_1',
 'e211_2',
 'e211_3',
 'e215_1',
 'e51_8',
 'e217_1',
 'e218_1',
 'e51_9',
 'e220_1',
 'e221_1',
 'e51_10',
 'e223_1',
 'e224_1',
 'e51_11',
 'e226_1',
 'e212_1',
 'e51_7',
 'e51_7_1',
 'e214_1',
 'E559_1',
 'E559_2',
 'E560_1',
 'E560_1_1',
 'E560_2',
 'E560_2_1',
 'E561_1',
 'F69_1',
 'F275_1',
 'F71_2',
 'F72_2',
 'F76_2',
 'F264_1',
 'F265_1',
 'F80_2',
 'F266_1',
 'F266_2',
 'F85_1',
 'F88_1',
 'F88_2',
 'F90_2',
 'F91_2',
 'F94_2',
 'F98_1',
 'F98_2',
 'F98_3',
 'F108_1',
 'F118_1',
 'F118_2',
 'F119_2',
 'F120_2',
 'F124_1',
 'F124_2',
 'F124_3',
 'F124_4',
 'F124_5',
 'G_ID_1',
 'G_ID_2',
 'G_ID_3',
 'G_ID_1A',
 'G_ID_2A',
 'G_ID_3A',
 'G126_1',
 'G126_2',
 'G126_3',
 'G126_4',
 'G126_5',
 'G126_6',
 'G126_7',
 'G126_8',
 'G250_1',
 'G250_2',
 'G250_5',
 'G250_3',
 'G250_4',
 'G127_1',
 'G127_2',
 'G127_3',
 'G128_1',
 'G129_1',
 'G129_2',
 'G130_1',
 'G131_1',
 'G132_1',
 'G132_2',
 'G132_3',
 'G133_1',
 'G133_2',
 'G_ST_1',
 'G134_1',
 'G134_2',
 'G134_3',
 'G134_4',
 'G134_5',
 'G134_6',
 'G134_7',
 'G134_8',
 'G251_1',
 'G251_2',
 'G251_5',
 'G251_3',
 'G251_4',
 'G135_1',
 'G135_2',
 'G135_3',
 'G136_1',
 'G137_1',
 'G137_2',
 'G138_1',
 'G139_1',
 'G140_1',
 'G140_2',
 'G140_3',
 'G141_1',
 'G141_2',
 'G_ITND_1',
 'G_ITND_2',
 'G144_1',
 'G144_2_1',
 'G144_2_2',
 'G144_2_3',
 'G144_2_4',
 'G144_2_5',
 'G_ITND_3',
 'G_IT_1',
 'G148_1_1',
 'G148_1_2',
 'G148_1_3',
 'G148_1_4',
 'G148_1_5',
 'G148_1_6',
 'G148_1_7',
 'G148_1_8',
 'G148_1_9',
 'G148_1_12',
 'G148_1_10',
 'G148_1_11',
 'G_IT_2',
 'G148_2_1',
 'G148_2_2',
 'G148_2_3',
 'G148_2_4',
 'G148_2_5',
 'G148_2_6',
 'G148_2_7',
 'G148_2_8',
 'G148_2_9',
 'G148_2_12',
 'G148_2_10',
 'G148_2_11',
 'G148_3',
 'G148_4',
 'G148_5_1',
 'G148_5_2',
 'G149_1',
 'G151_6',
 'G151_3',
 'G151_4',
 'G153_1',
 'G153_2',
 'G258_1','E247',
 'G154_1',
 'tipo_ocupado','F98',
 'mto_cuota',
 'mto_emer',
 'mto_hogcon', 'G257',
 'mto_desay',
 'mto_almue','E27','F70','F85','F87','G142','G143','G145','G146','G147','SUBEMPLEO',
 'mto_vacas',
 'mto_oveja','lecheenpol'
 'mto_caball','YSVL','nper','E558','E27','F70','F113',
 'li_06','H164','H165','H166','I174','I175','HT11','HT13','ht19','YHOG','E27',
 'lp_06'])
#noobj = [(column,df[column].dtype) for column in df.columns if df[column].dtype != np.dtype('O') and column not in excluded]
#print("Faltan {} columnas".format(len(noobj)))
#print(noobj)
#for tuplet in noobj:
#    print(tuplet[0])

In [96]:
new_dict = {'ANIO': 'AÑO DE LA ENCUESTA',
 'DPTO': 'DEPARTAMENTO',
 'NOMDPTO': 'NOMBRE DEL DEPARTAMENTO',
 'SECC': 'SECCIÓN CENSAL',
 'SEGM': 'SEGMENTO CENSAL',
 'CCZ': 'CENTRO COMUNAL ZONAL',
 'REGION_3': 'REGION_3',
 'REGION_4': 'REGION_4',
 'PESOMEN': 'PONDERADOR DEL MES',
 'H5_CV': 'TENENCIA DE LA VIVIENDA',
 'D23': 'PERSONAS DE 14 O MÁS AÑOS',
 'D24': 'MENORES DE 14 AÑOS',
 'D25': 'TOTAL DE PERSONAS DEL HOGAR',
 'H155': 'RECIBE DINERO DE ALGÚN FAMILIAR U OTRO HOGAR DEL PAÍS',
 'H156': 'RECIBE AYUDA EN ESPECIE DE ALGÚN FAMILIAR U OTRO HOGAR DEL PAÍS',
 'H252': 'HOGAR ALQUILA PARTE DE LA VIVIENDA QUE HABITA',
 'H159': 'PROPIETARIO DE OTRA VIVIENDA O LOCAL',
 'H160': 'FUERON ALQUILADAS SUS PROPIEDADES',
 'H161': 'PROPIETARIO DE CHACRA O CAMPO EN EL QUE NO TRABAJA',
 'H162': 'FUERON ARRENDADOS SU CHACRA QUE NO TRABAJA',
 'H164': 'INGRESO POR MEDIANERÍA SIN HABER TRABAJADO',
 'H165': 'INGRESO POR PASTOREO SIN HABER TRABAJADO',
 'H227': 'PROPIETARIO DE GANADO A CAPITALIZACIÓN',
 'H166': 'INGRESO POR GANADO A CAPITALIZACIÓN',
 'H269': 'RECIBIÓ INGRESOS POR ALQUILER DE VEHÍCULOS / MAQUINARIA',
 'H167_1': 'DEPÓSITOS BANCARIOS',
 'H167_1_1': 'MONTO PERCIBIDO POR INTERESES BANCARIOS DEL PAÍS',
 'H167_2': 'LETRAS O BONOS',
 'H167_3': 'PRÉSTAMOS A TERCEROS',
 'H167_4': 'OTROS TÍTULOS',
 'H169': 'PROPIETARIO DE NEGOCIO EN EL QUE NO TRABAJA',
 'H271': 'RECIBIÓ INGRESOS POR MARCAS, PATENTES, DERECHOS DE AUTOR',
 'H171': 'RECIBIÓ INDEMNIZACIÓN POR DESPIDO',
 'H172': 'RECIBIÓ ALGUNA COLABORACIÓN ECONÓMICA DE ALGÚN FAMILIAR DEL EXTERIOR',
 'H173': 'RECIBIÓ ALGÚN INGRESO EXTRAORDINARIO',
 'I228': 'CONTRIBUCIONES POR DIVORCIO O SEPARACIÓN',
 'I259': 'CONTRIBUCIONES EN DINERO O EN ESPECIE A OTROS HOGARES',
 'HT11': 'INGRESO TOTAL DEL HOGAR CON VALOR LOCATIVO SIN SERVICIO DOMÉSTICO',
 'HT13': 'VALOR LOCATIVO',
 'HT19': 'CANTIDAD DE PERSONAS SIN SERVICIO DOMÉSTICO',
 'YHOG': 'INGRESOS DEL HOGAR NO IMPUTABLES A PERSONAS',
 'YSVL': 'INGRESO TOTAL DEL HOGAR SIN VALOR LOCATIVO SIN SERVICIO DOMÉSTICO',
 'LP_06': 'LÍNEA DE POBREZA',
 'LI_06': 'LÍNEA DE INDIGENCIA',
 'POBRE06': 'POBREZA SEGÚN METODOLOGÍA 2006',
 'INDIGENTE06': 'INDIGENCIA SEGÚN METODOLOGÍA 2006',
 'NPER': 'NUMERO DE PERSONA',
 'HOGAR': 'HOGAR',
 'MES': 'MES DE LA ENCUESTA',
 'LOC_AGR_13': 'LOCALIDAD AGRUPADA',
 'NOM_LOC_AGR_13': 'NOMBRE DE LOCALIDAD',
 'NOMBARRIO': 'NOMBRE DEL BARRIO',
 'ESTRED13': 'ESTRATO',
 'E0_CV': 'NUEVO MIEMBRO DEL HOGAR',
 'E01_CV': 'NUEVO MIEMBRO DEL HOGAR DEPENDE DEL MISMO FONDO DE ALIMENTACIÓN',
 'E1_CV': 'PERSONA CONTINUA RESIDIENDO EN EL HOGAR',
 'E557': 'IDENTIFICACIÓN DEL INFORMANTE',
 'E558': 'NUMERO DE PERSONA',
 'E26': 'SEXO',
 'E27': 'EDAD',
 'E29_1': 'AFRO O NEGRA',
 'E29_2': 'ASIÁTICA O AMARILLA',
 'E29_3': 'BLANCA',
 'E29_4': 'INDÍGENA',
 'E29_5': 'OTRA',
 'E29_6': 'PRINCIPAL ASCENDENCIA',
 'E30': 'RELACIÓN DE PARENTESCO',
 'E45_CV': 'INSTITUCIÓN EN DONDE CUENTA CON DERECHOS PARA ATENDER SU SALUD',
 'E45_1_1_CV': 'MECANISMO ACCESO MSP/ASSE',
 'E45_2_1_CV': 'MECANISMO ACCESO IAMC',
 'E45_3_1_CV': 'MECANISMO ACCESO A SEGURO PRIVADO MÉDICO',
 'E45_4_1_CV': 'MECANISMO ACCESO A HOSPITAL POLICIAL/MILITAR',
 'E46_CV': 'AFILIACIÓN A EMERGENCIA MÓVIL',
 'E47_CV': 'QUIÉN PAGA LA CUOTA DE LA EMERGENCIA MÓVIL',
 'E49_CV': 'ASISTENCIA A UN ESTABLECIMIENTO DE ENSEÑANZA',
 'E50_CV': 'TIPO DE CENTRO EDUCATIVO',
 'E196_CV': 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'E196_1_CV': 'DESAYUNOS Y/O MERIENDAS EN CENTRO EDUCATIVO',
 'E562_CV': 'BECAS',
 'E48': 'SABE LEER Y ESCRIBIR',
 'E49': 'ASISTENCIA ALGUNA VEZ A UN ESTABLECIMIENTO DE ENSEÑANZA',
 'E238': 'ASISTENCIA A EDUCACIÓN INICIAL',
 'E239': 'TIPO DE CENTRO EDUCATIVO',
 'E240_1': 'CANTIDAD DE DÍAS QUE ASISTE A LA SEMANA',
 'E240_2': 'CANTIDAD DE HORAS AL DÍA',
 'E241': 'FALTÓ ALGÚN DÍA',
 'E242': 'RAZONES POR LAS QUE NO ASISTIÓ',
 'E193': 'ASISTENCIA A EDUCACIÓN PREESCOLAR',
 'E194': 'TIPO DE CENTRO EDUCATIVO',
 'E243_1': 'CANTIDAD DE DÍAS QUE ASISTE A LA SEMANA',
 'E243_2': 'CANTIDAD DE HORAS AL DÍA',
 'E244': 'FALTÓ ALGÚN DÍA',
 'E245': 'RAZONES POR LAS QUE NO ASISTIÓ',
 'E196': 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'E196_1': 'DESAYUNOS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
 'E196_2': 'ALMUERZOS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
 'E196_3': 'CENAS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
 'E197': 'ASISTENCIA A EDUCACIÓN PRIMARIA',
 'E197_1': 'FINALIZÓ EL NIVEL',
 'E51_2': 'AÑOS APROBADOS EN PRIMARIA COMÚN',
 'E51_3': 'AÑOS APROBADOS EN PRIMARIA ESPECIAL',
 'E198': 'TIPO DE CENTRO EDUCATIVO',
 'E199': 'TIPO DE CENTRO EDUCATIVO',
 'E200': 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'E200_1': 'DESAYUNOS SEMANALES',
 'E200_2': 'ALMUERZOS SEMANALES',
 'E200_3': 'CENAS SEMANALES',
 'E201': 'ASISTENCIA A EDUCACIÓN MEDIA',
 'E201_1': 'FINALIZÓ EL NIVEL',
 'E202_1': 'ABANDONO EDUCACIÓN POR COMENZÓ A TRABAJAR',
 'E202_9': 'ABANDONO EDUCACIÓN POR NO TENÍA INTERÉS',
 'E202_10': 'ABANDONO EDUCACIÓN POR LE INTERESABA APRENDER OTRAS COSAS',
 'E202_3': 'ABANDONO EDUCACIÓN POR QUEDÓ USTED O SU PAREJA EMBARAZADA',
 'E202_4': 'ABANDONO EDUCACIÓN POR LE RESULTABAN DIFÍCILES LA/S MATERIA/S',
 'E202_8': 'ABANDONO EDUCACIÓN POR POR DIFICULTADES ECONÓMICAS (COMPRA DE MAT. DE ESTUDIO, ETC.)',
 'E202_5': 'ABANDONO EDUCACIÓN POR PORQUE TUVO QUE ATENDER ASUNTOS FAMILIARES',
 'E202_6': 'ABANDONO EDUCACIÓN POR OTRAS RAZONES',
 'E202_6_1': 'ABANDONO EDUCACIÓN POR DESCRIPCIÓN OTRA RAZÓN DE NO FINALIZACIÓN',
 'E202_7': 'PRINCIPAL RAZÓN DE NO FINALIZACIÓN',
 'E51_4': 'AÑOS APROBADOS EN CICLO BÁSICO',
 'E210_1': 'TIPO DE CENTRO EDUCATIVO',
 'E51_5': 'AÑOS APROBADOS EN BACHILLERATO',
 'E210_2': 'TIPO DE CENTRO EDUCATIVO',
 'E51_6': 'AÑOS APROBADOS EN BACHILLERATO TECNOLÓGICO',
 'E209_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E210_3': 'TIPO DE CENTRO EDUCATIVO',
 'E211': 'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
 'E211_1': 'CANTIDAD DE COMIDAS SEMANALES',
 'E211_2': 'CANTIDAD ALMUERZOS SEMANALES (EST. BACHILLERATO)',
 'E562': 'BECAS',
 'E212': 'ASISTENCIA A EDUCACIÓN TÉCNICA',
 'E212_1': 'FINALIZÓ EL NIVEL',
 'E213': 'TIPO DE CENTRO EDUCATIVO',
 'E51_7': 'AÑOS APROBADOS EN EDUCACIÓN TÉCNICA',
 'E51_7_1': 'EXIGENCIA PARA REALIZAR CURSO DE EDUCACIÓN TÉCNICA',
 'E214_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E215': 'ASISTENCIA A MAGISTERIO O PROFESORADO',
 'E215_1': 'FINALIZÓ EL NIVEL',
 'E216': 'TIPO DE CENTRO EDUCATIVO',
 'E51_8': 'AÑOS APROBADOS EN MAGISTERIO O PROFESORADO',
 'E217_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E218': 'ASISTENCIA A UNIVERSIDAD O SIMILAR',
 'E218_1': 'FINALIZÓ EL NIVEL',
 'E219': 'TIPO DE CENTRO EDUCATIVO',
 'E51_9': 'AÑOS APROBADOS EN UNIVERSIDAD O SIMILAR',
 'E220_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E221': 'ASISTENCIA A TERCIARIO NO UNIVERSITARIO',
 'E221_1': 'FINALIZÓ EL NIVEL',
 'E222': 'TIPO DE CENTRO EDUCATIVO',
 'E51_10': 'AÑOS APROBADOS EN TERCIARIO NO UNIVERSITARIO',
 'E223_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E224': 'ASISTENCIA A POSGRADO',
 'E224_1': 'FINALIZÓ EL NIVEL',
 'E225': 'TIPO DE CENTRO EDUCATIVO',
 'E51_11': 'AÑOS APROBADOS EN POSGRADO',
 'E226_1': 'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
 'E559': 'CONCURRENCIA A COMEDOR',
 'E559_1': 'DESAYUNOS Y/O MERIENDAS RECIBIDAS',
 'E559_2': 'ALMUERZOS Y/O CENAS RECIBIDAS',
 'E560': 'RECIBE ALGÚN TIPO DE TARJETA',
 'E560_1': 'TUS-MIDES',
 'E560_1_1': 'CANTIDAD RECIBIDA EL MES ANTERIOR (TUS-MIDES)',
 'E560_2': 'TUS-INDA',
 'E560_2_1': 'CANTIDAD RECIBIDA EL MES ANTERIOR (TUS-INDA)',
 'E561': 'LECHE EN POLVO',
 'E561_1': 'CANTIDAD DE LECHE EN POLVO',
 'E59': 'RECIBE ALGÚN TIPO DE CANASTA',
 'E246': 'TIPO DE CANASTA',
 'E247': 'CANTIDAD DE CANASTAS MENSUALES',
 'F269': 'TRABAJÓ LA SEMANA PASADA',
 'F270': 'COLABORÓ EN NEGOCIO FAMILIAR O TRABAJO CONCRETO LA SEMANA PASADA',
 'F271': 'RECIBIÓ SUELDO',
 'F272': 'TRABAJO REALIZADO PARA UN NEGOCIO DEL HOGAR',
 'F273': 'NEGOCIO O ACTIVIDAD POR LA QUE PERCIBE UN PAGO',
 'F69': 'MOTIVO POR EL CUAL NO TRABAJÓ',
 'F69_1': 'FUE DESPEDIDO DE SU PUESTO DE TRABAJO',
 'F274': 'RECIBIÓ SUELDO',
 'F275': 'TIEMPO EN EL QUE VOLVERÁ A TRABAJAR',
 'F275_1': 'SI TERMINARAN RESTRICCIONES POR PANDEMIA, CREE VOLVER A SU PUESTO DE TRABAJO',
 'F276': 'REALIZÓ TAREAS AGRÍCOLAS PARA CONSUMO DEL HOGAR',
 'F277': 'DE LO QUE PRODUCE PARTE SE DESTINA A LA VENTA',
 'F70': 'TRABAJOS QUE TIENE',
 'F71_2': 'TAREAS QUE PROPORCIONAN MAYORES INGRESOS',
 'F72_2': 'ACTIVIDAD DEL ESTABLECIMIENTO DONDE REALIZA LAS TAREAS',
 'F73': 'CATEGORÍA DE LA OCUPACIÓN',
 'F74': 'SECTOR PÚBLICO',
 'F75': 'ESTABLECIMIENTO DONDE REALIZA LAS TAREAS PERTENECE A LA EMPRESA QUE LO CONTRATA',
 'F76_2': 'A QUÉ SE DEDICA LA EMPRESA QUE LO CONTRATA',
 'F262': 'NEGOCIO CON CONTABILIDAD COMPLETA ',
 'F263': 'NEGOCIO REGISTRADO ANTE LAS OFICINAS DE IMPUESTOS O DE LA SEGURIDAD SOCIAL',
 'F264': 'NATURALEZA JURÍDICA DEL NEGOCIO',
 'F265': 'FORMA DE TRIBUTACIÓN',
 'F77': 'TAMAÑO DE LA EMPRESA ',
 'F78': 'DÓNDE REALIZA SU TRABAJO',
 'F80': 'TRABAJO EN EL DEPARTAMENTO',
 'F81': 'COBRA AGUINALDO',
 'F82': 'APORTE A CAJA DE JUBILACIONES',
 'F83': 'CAJA A LA CUAL APORTA',
 'F84': 'APORTE POR TOTALIDAD DEL SALARIO',
 'F266': 'RECONOCIMIENTO DE HORAS EXTRAS MEDIANTE PAGO, COMPENSACIÓN, DÍAS ADICIONALES DE VACACIONES',
 'F266_1': 'COMO PAGAN LAS HORAS EXTRAS',
 'F266_2': 'HORARIO FLEXIBLE',
 'F267': 'VACACIONES ANUALES PAGAS O PAGO LICENCIA NO GOZADA',
 'F268': 'LICENCIA POR ENFERMEDAD O LESIÓN PAGA',
 'F85': 'HORAS TRABAJADAS POR SEMANA',
 'F85_1': 'HORAS EFECTIVAS DE TRABAJO',
 'F86': 'TIENE QUE LLEVAR TRABAJO PARA SU HOGAR (SOLO PARA ASALARIADOS)',
 'F87': 'CANTIDAD DE HORAS SEMANALES DEDICADAS A ESE TRABAJO EN EL HOGAR (SOLO PARA ASALARIADOS)',
 'F88_1': 'TIEMPO QUE TRABAJA EN FORMA CONTINUA',
 'F89': 'ESTUVO DESOCUPADO EN LOS ÚLTIMOS 12 MESES',
 'F90_2': 'TAREAS QUE REALIZA EN EL OTRO TRABAJO',
 'F91_2': 'A QUÉ SE DEDICA EL ESTABLECIMIENTO DONDE REALIZA LAS TAREAS',
 'F92': 'CATEGORÍA DE LA OCUPACIÓN',
 'F93': 'TAMAÑO DE LA EMPRESA ',
 'F94': 'TRABAJO EN EL DEPARTAMENTO',
 'F95': 'COBRA AGUINALDO',
 'F9': 'APORTE A CAJA DE JUBILACIONES',
 'F97': 'CAJA A LA CUAL APORTA',
 'F98': 'HORAS TRABAJADAS HABITUALMENTE EN OTRAS OCUPACIONES',
 'F98_1': 'HORAS EFECTIVAS TRABAJADAS EN OTRAS OCUPACIONES',
 'F98_2': 'TELETRABAJO HABITUAL',
 'F98_3': 'REALIZÓ TELETRABAJO',
 'F99': 'BUSCA OTRO TRABAJO',
 'F100': 'SUSTITUIR O COMPLEMENTAR EL ACTUAL',
 'F101': 'RAZONES POR LAS CUALES QUIERE OTRO EMPLEO',
 'F102': 'DESEA TRABAJAR MÁS HORAS',
 'F103': 'ESTÁ DISPONIBLE PARA TRABAJAR MÁS HORAS',
 'F104': 'RAZONES POR LAS CUALES NO TRABAJA MÁS HORAS A LA SEMANA',
 'F105': 'QUÉ HIZO PARA TRABAJAR MÁS HORAS',
 'F106': 'DISPONIBILIDAD PARA TRABAJAR',
 'F107': 'LA SEMANA PASADA BUSCÓ TRABAJO',
 'F108': 'RAZONES POR LAS CUALES NO BUSCÓ TRABAJO NI ESTABLECIÓ SU NEGOCIO',
 'F109': 'BUSCÓ TRABAJO EN LAS ÚLTIMAS CUATRO SEMANAS',
 'F110': 'QUÉ HIZO PARA BUSCAR TRABAJO O ESTABLECER SU PROPIO NEGOCIO',
 'F111': 'BUSCA TRABAJO CON CONDICIONES ESPECIALES',
 'F112': 'PRINCIPAL CONDICIÓN QUE REQUIERE',
 'F113': 'TIEMPO QUE BUSCA TRABAJO',
 'F114': 'REALIZÓ GESTIONES CONCRETAS',
 'F115': 'REALIZÓ ALGÚN TRABAJO OCASIONAL',
 'F116': 'HA TRABAJADO ANTES',
 'F117': 'ESTÁ EN SEGURO DE DESEMPLEO',
 'F118_1': 'TIEMPO QUE HACE QUE DEJÓ EL ÚLTIMO EMPLEO (MESES)',
 'F119_2': 'TAREAS QUE REALIZABA EN ESA OCUPACIÓN',
 'F120_2': 'A QUÉ SE DEDICABA O QUÉ PRODUCÍA EL ESTABLECIMIENTO EN QUE REALIZABA SUS TAREAS',
 'F121': 'CATEGORÍA DE LA OCUPACIÓN',
 'F122': 'RAZONES POR LAS CUALES DEJÓ EL TRABAJO',
 'F123': 'APORTABA A CAJA DE JUBILACIONES',
 'F124_1': 'JUBILADO/A',
 'F124_2': 'PENSIONISTA',
 'F124_3': 'RENTISTA',
 'F124_4': 'ESTUDIANTE',
 'F124_5': 'QUIEN REALIZA PRINCIPALMENTE LOS QUEHACERES DEL HOGAR',
 'F125': 'TIPO DE PENSIÓN QUE RECIBE',
 'G_ID_1': 'SUELDOS, JORNALES, AGUINALDO O SALARIO VACACIONAL POR ALGÚN TRABAJO ANTERIOR',
 'G_ID_2': 'COMISIONES, INCENTIVOS, PAGO DE HORAS EXTRAS, HABILITACIONES U OTRO INGRESO POR ALGÚN TRABAJO ANTERIOR',
 'G_ID_3': 'ALIMENTOS, BEBIDAS O ALGÚN OTRO TIPO DE RETRIBUCIÓN EN ESPECIE POR ALGÚN TRABAJO ANTERIOR',
 'G_ID_1A': 'SUELDOS, JORNALES, AGUINALDO O SALARIO VACACIONAL POR ALGÚN TRABAJO ANTERIOR EN RELACIÓN DE DEPENDENCIA',
 'G_ID_2A': 'COMISIONES, INCENTIVOS, PAGO DE HORAS EXTRAS, HABILITACIONES U OTRO INGRESO POR ALGÚN TRABAJO ANTERIOR EN RELACIÓN DE DEPENDENCIA',
 'G_ID_3A': 'ALIMENTOS, BEBIDAS O ALGÚN OTRO TIPO DE RETRIBUCIÓN EN ESPECIE POR ALGÚN TRABAJO ANTERIOR EN RELACIÓN DE DEPENDENCIA',
 'G126_1': 'SUELDO O JORNALES LÍQUIDOS',
 'G126_2': 'COMISIONES, INCENTIVOS, HORAS EXTRAS, HABILITACIONES',
 'G126_3': 'VIÁTICOS NO SUJETOS A RENDICIÓN',
 'G126_4': 'PROPINAS',
 'G126_5': 'AGUINALDO',
 'G126_6': 'SALARIO VACACIONAL',
 'G126_7': 'PAGOS ATRASADOS',
 'G126_8': 'BOLETOS DE TRANSPORTE',
 'G250_1': 'COBRA EN EFECTIVO / CHEQUE',
 'G250_2': 'COBRA POR DEPÓSITO EN CUENTA BANCARIA',
 'G250_3': 'COBRA POR GIROS O TRANSFERENCIAS BANCARIAS',
 'G250_4': 'COBRA POR GIROS O TRANSFERENCIAS NO BANCARIAS',
 'G250_5': 'COBRA TARJETA PREPAGA (DEANDA, MIDINERO, ETC.)',
 'G127': 'RECIBIÓ ALIMENTOS O BEBIDAS',
 'G127_1': 'RECIBIO  DESAYUNOS / MERIENDAS',
 'G127_2': 'RECIBIO ALMUERZOS / CENAS',
 'G127_3': ' MONTO ESTIMADO RECIBIDO DE ALIMENTOS',
 'G128': 'RECIBIÓ TIQUES DE ALIMENTACIÓN',
 'G128_1': 'VALOR RECIBIDO EL MES PASADO',
 'G129': 'RECIBIÓ VIVIENDA O ALOJAMIENTO',
 'G129_1': 'ESTÁ EN EL MEDIO URBANO',
 'G129_2': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESE ALOJAMIENTO',
 'G130': 'RECIBIÓ OTRO TIPO DE RETRIBUCIÓN EN ESPECIE',
 'G130_1': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS BIENES',
 'G131': 'RECIBIÓ ALGÚN OTRO TIPO DE COMPLEMENTO PAGADO POR EL EMPLEADOR',
 'G131_1': 'MONTO PERCIBIDO COMPLEMENTO PAGADOR',
 'G132': 'DERECHO A PASTOREO TRABAJO PRINCIPAL',
 'G132_1': 'CANTIDAD DE VACUNOS',
 'G132_2': 'CANTIDAD DE OVINOS',
 'G132_3': 'CANTIDAD DE EQUINOS',
 'G133': 'DERECHO A CULTIVAR PARA CONSUMO PROPIO',
 'G133_1': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS PRODUCTOS',
 'G133_2': 'MONTO PERCIBIDO POR LA VENTA DE ESOS PRODUCTOS',
 'G_ST_1': 'INGRESOS DEL MES ANTERIOR POR ALGÚN OTRO EMPLEO ASALARIADO',
 'G134_1': 'SUELDOS O JORNALES LÍQUIDOS',
 'G134_2': 'COMISIONES, INCENTIVOS, HORAS EXTRAS, HABILITACIONES',
 'G134_3': 'VIÁTICOS NO SUJETOS A RENDICIÓN',
 'G134_4': 'PROPINAS',
 'G134_5': 'AGUINALDO',
 'G134_6': 'SALARIO VACACIONAL',
 'G134_7': 'PAGOS ATRASADOS',
 'G134_8': 'BOLETOS DE TRANSPORTE',
 'G251_1': 'EN EFECTIVO / CHEQUE',
 'G251_2': 'POR DEPÓSITO EN CUENTA BANCARIA',
 'G251_3': 'POR GIROS O TRANSFERENCIAS BANCARIAS',
 'G251_4': 'POR GIROS O TRANSFERENCIAS NO BANCARIAS',
 'G251_5': 'TARJETA PREPAGA (DEANDA, MIDINERO, ETC.)',
 'G135': 'RECIBIÓ ALIMENTOS O BEBIDAS',
 'G135_1': 'CANTIDAD DE DESAYUNOS / MERIENDAS RECIBIDAS',
 'G135_2': 'CANTIDAD DE ALMUERZOS / CENAS RECIBIDAS',
 'G135_3': 'OTROS - MONTO ESTIMADO RECIBIDAS',
 'G136': 'RECIBIÓ TIQUES DE ALIMENTACIÓN',
 'G136_1': 'MONTO TICKET ALIMENTACION',
 'G137': 'RECIBIÓ VIVIENDA O ALOJAMIENTO',
 'G137_1': 'MEDIO URBANO',
 'G137_2': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESE ALOJAMIENTO',
 'G138': 'RECIBIÓ OTRO TIPO DE RETRIBUCIÓN EN ESPECIE',
 'G138_1': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS BIENES DE OTROS BIENES',
 'G139': 'RECIBIÓ ALGÚN OTRO COMPLEMENTO PAGADO POR EL EMPLEADOR',
 'G139_1': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR  EL COMPLEMENTO EMPLEADOR',
 'G140': 'DERECHO A PASTOREO',
 'G141': 'DERECHO A CULTIVAR ',
 'G_ITND_1': 'RETIRÓ DINERO PARA GASTOS DEL HOGAR DE NEGOCIOS QUE TIENE O TENÍA',
 'G_ITND_2': 'INGRESOS POR DISTRIBUCIÓN DE UTILIDADES POR NEGOCIO QUE TIENE O TENÍA',
 'G144': 'RETIRO DE PRODUCTOS PARA CONSUMO PROPIO (TRABAJADOR NO AGROPECUARIO)',
 'G144_2_1': 'VALOR DE LO CONSUMIDO EN CARNES O CHACINADOS',
 'G_ITND_3': 'RECIBIÓ INGRESOS POR MEDIANERÍA O APARCERÍA',
 'G145': 'INGRESO POR MEDIANERÍA O APARCERÍA',
 'G146': 'INGRESO POR PASTOREO',
 'G147': 'INGRESO POR GANADO A CAPITALIZACIÓN',
 'G_IT_1': 'RECIBIÓ INGRESOS POR JUBILACIONES',
 'G148_1_1': 'MONTO APORTES DE BPS - CAJA INDUSTRIA Y COMERCIO',
 'G148_1_2': 'MONTO APORTES DEBPS - CAJA CIVIL Y ESCOLAR',
 'G148_1_3': 'MONTO APORTES BPS - RURAL Y SERVICIO DOMÉSTICO',
 'G148_1_4': 'MONTO APORTES UNIÓN POSTAL',
 'G148_1_5': 'MONTO APORTES POLICIAL',
 'G148_1_6': 'MONTO APORTES MILITAR',
 'G148_1_7': 'MONTO APORTES CAJA PROFESIONAL',
 'G148_1_8': 'MONTO APORTES CAJA NOTARIAL',
 'G148_1_9': 'MONTO APORTES CAJA BANCARIA',
 'G148_1_12': 'MONTO APORTES AFAP (INGRESOS PROVENIENTES ÚNICAMENTE DE ALGUNA AFAP)',
 'G148_1_10': 'MONTO APORTES OTRA CAJA',
 'G148_1_11': 'MONTO APORTES OTRO PAÍS',
 'G_IT_2': 'RECIBIÓ INGRESOS POR PENSIONES',
 'G148_2_1': 'BPS - CAJA INDUSTRIA Y COMERCIO',
 'G148_2_2': 'BPS - CAJA CIVIL Y ESCOLAR',
 'G148_2_3': 'BPS - RURAL Y SERVICIO DOMÉSTICO',
 'G148_2_4': 'UNIÓN POSTAL',
 'G148_2_5': 'POLICIAL',
 'G148_2_6': 'MILITAR',
 'G148_2_7': 'PROFESIONAL',
 'G148_2_8': 'NOTARIAL',
 'G148_2_9': 'BANCARIA',
 'G148_2_12': 'AFAP (INGRESOS PROVENIENTES ÚNICAMENTE DE ALGUNA AFAP)',
 'G148_2_10': 'OTRA',
 'G148_2_11': 'OTRO PAÍS',
 'G148_3': 'SEGURO DE DESEMPLEO',
 'G148_4': 'COMPENSACIONES POR ACCIDENTE, MATERNIDAD O ENFERMEDAD',
 'G148_5_1': 'BECAS, SUBSIDIOS, DONACIONES',
 'G149': 'COBRA HOGAR CONSTITUIDO',
 'G150': 'COBRA ASIGNACIONES FAMILIARES',
 'G255': 'RECIBE LAS ASIGNACIONES POR EL PLAN DE EQUIDAD DEL MIDES',
 'G256': 'FORMA DE COBRO DE LAS ASIGNACIONES FAMILIARES',
 'G152': 'FRECUENCIA DE COBRO DE ASIGNACIONES FAMILIARES',
 'G151_6': 'CANTIDAD DE ASIGNACIONES FAMILIARES',
 'G257': 'MONTO PERCIBIDO POR ASIGNACIONES FAMILIARES',
 'G153': 'RECIBE PENSIÓN ALIMENTICIA O ALGUNA CONTRIBUCIÓN POR DIVORCIO O SEPARACIÓN',
 'G258': 'DEVOLUCIÓN POR FONASA',
 'G154': 'OTRO INGRESO CORRIENTE ADEMÁS DE LOS DECLARADOS',
 'POBPCOAC': 'CONDICIÓN DE ACTIVIDAD ECONÓMICA',
 'SUBEMPLEO': 'TRABAJADOR SUBEMPLEADO ',
 'TIPO_OCUPADO': 'TIPO DE OCUPADO',
 'PT1': 'TOTAL DE INGRESOS PERSONALES',
 'PT2': 'INGRESOS DE LA OCUPACIÓN PRINCIPAL',
 'PT4': 'TOTAL DE INGRESOS POR TRABAJO',
 'H155_1': 'MONTO QUE RECIBIÓ EL MES PASADO DE FAMILIAR EN EL PAIS',
 'H156_1': 'MONTO ESTIMADO DE LA AYUDA EN ESPECIEQUE RECIBIÓ EL MES PASADO',
 'H160_1': 'MONTO RECIBIDO POR ALQUILER INTERIOR',
 'H160_2': 'MONTO RECIBIDO POR ALQUILER FUERA DEL PAIS',
 'H158_1': 'PROPIETARIO DE VIVIENDA 1',
 'H158_2': 'PROPIETARIO DE VIVIENDA 2',
 '163_1': 'MONTO RECIBIDO POR ARRENDAMIENTO DE CAMPO EN EL PAIS',
 '162_2': 'MONTO RECIBIDO POR ARRENDAMIENTO DE CAMPO FUERA DEL PAIS',
 'H269_1': 'MONTO RECIBIDO POR ALQUILER DE VEHICULOS',
 'H167_1_2': 'MONTO PERCIBIDO POR INTERESES BANCARIOS DEL EXTRANJERO',
 'H167_3_1': 'MONTO PERCIBIDO POR INTERESES DEL PAÍS',
 'H167_3_2': 'MONTO PERCIBIDO POR INTERESES DE PRESTAMOS A  TERCEROS DEL EXTRANJERO',
 'H167_4_1': 'MONTO PERCIBIDO POR OTROS TITULO SEN EL PAÍS',
 'H167_4_2': 'MONTO PERCIBIDO POR INTERESES POR OTROS TITULOS DEL EXTRANJERO',
 'H163_1': 'MONTO PERCIBIDO POR ARRENDAMIENTOS DE CHACRA EN EL PAÍS',
 'H163_2': 'MONTO PERCIBIDO POR ARRENDAMIENTOS DE CHACRA DEL EXTRANJERO',
 'H271_1': 'INGRESOS POR PATENTES O DERECHOS DE AUTOR',
 'H170_1': 'MONTO  QUE PERCIBE POR INGRESO DE NEGOCIO EN QUE NO TRABAJA EN URUGUAY',
 'H170_2': 'MONTO  QUE PERCIBE POR INGRESO DE NEGOCIO EN QUE NO TRABAJA EN EL EXTRANJERO',
 'I174': 'MONTOS POR CONTRIBUCIONES POR DIVORCIO O SEPARACION',
 'I175': 'MONTO DE CONTRIBUCIONES EN DINERO O EN ESPECIE A OTROS HOGARES',
 'H167_2_1': 'MONTO PERCIBIDO POR INTERESES DE LETRAS DEL PAÍS',
 'H167_2_2': 'MONTO PERCIBIDO POR INTERESES DE LETRAS DEL EXTRANJERO',
 'E196_2_CV': 'ALMUERZOS / CENAS EN CENTRO EDUCATIVO',
 'E196_3_CV': 'MERIENDAS EN CENTRO EDUCATIVO',
 'H252_1': 'MONTO RECIBIDO POR ALQUILER DE PARTE DE LA VIVIENDA',
 'H171_1': 'MONTO RECIBIDO POR DESPIDO',
 'H172_1': 'MONTO DE COLABORACIÓN ECONÓMICA DE ALGÚN FAMILIAR DEL EXTERIOR',
 'H173_1': 'MONTO DEL INGRESO EXTRAORDINARIO RECIBIDO',
 'E245_1': 'DESCRIPCIÓN OTRA RAZÓN DE NO ASISTENCIA',
 'E211_3': 'MERIENDAS RECIBIDAS EN BACHILLERATO',
 'F88_2': 'AÑOS QUE HACE DESDE QUE TRABAJA',
 'F96': 'APORTE CAJA JUBILACIONES',
 'G140_1': 'VACUNOS PASTOREANDO',
 'G140_2': 'OVINOS PASTOREANDO',
 'G140_3': 'EQUINOS PASTOREANDO',
 'G141_1': 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR CULTIVOS SI LOS TUVIERA QUE COMPRAR',
 'G141_2': 'MONTO PERCIBIDO POR LA VENTA DE ESOS PRODUCTOS QUE CULTIVO',
 'G142': 'MONTO DE DINERO QUE RETIRÓ  PARA GASTOS DEL HOGAR DE NEGOCIOS QUE TIENE O TENÍA',
 'G143': 'INGRESOS POR DISTRIBUCIÓN DE UTILIDADES POR NEGOCIO QUE TIENE O TENÍA',
 'G144_2_2': 'VALOR DE LO CONSUMIDO EN LÁCTEOS',
 'G144_2_3': 'VALOR DE LO CONSUMIDO EN HUEVOS Y AVES',
 'G144_2_4': 'VALOR DE LO CONSUMIDO EN PRODUCTOS DE LA HUERTA',
 'G144_2_5': 'VALOR CONSUMIDO EN OTROS ALIMENTOS',
 'G148_5_2': 'BECAS, SUBSIDIOS, DONACIONES DESDE EL EXTRANJERO',
 'G149_1': 'COBRA HOGAR CONSTITUIDO DECLARADO EN EL SUELDO',
 'G151_3': 'CANTIDAD DE ASIGNACIONES DE MENORES QUE ESTAN EN SECUNDARIA',
 'G151_4': 'CANTIDAD DE ASIGNACIONES A PERSONAS CON DISCAPACIDAD',
 'G153_1': 'RECIBE PENCION ALIMENTICIA POR DIVORCIO EN EL PAÍS',
 'G153_2': 'RECIBE PENCION ALIMENTICIA POR DIVORCIO EN EL EXTERIOR',
 'G258_1': 'MONTO DEVOLUCION FONASA',
 'G144_1': 'MONTO DE RETIRO DE PRODUCTOS PARA CONSUMO PROPIO (TRABAJADOR NO AGROPECUARIO)'}

In [97]:
#Recombra las variables
df.rename(columns=new_dict,inplace=True)
char_rep = [("\u2019","'"),("\u2020","'"),(r'\u0192','f')] #elimina errores detectados de encoding en los datos (aparecían cuando se salvaba el archivo en formato .csv)
for pair in char_rep:
        df = df.replace(pair[0],pair[1],regex=True)

#### Funcionalidades utilitarias para categorizar las variables

Se generaron funciones utilitarias para ayudar a la taera de categorizar las diferentes variables en el dataset. 

In [98]:
def get_stored(dict_columns):
    """Entrega una lista de variables que ya fueron categorizadas"""
    stored = []
    for value in dict_columns.values():
        for column in value:
            stored.append(column)
    return stored

def get_not_stored(dict_columns):
    """Entrega una iterable de variables que ya faltan categorizadar"""
    stored = get_stored(dict_columns)  
    not_stored = iter([x for x in df.columns if x not in stored])
    print("Faltan {}".format(len([x for x in df.columns if x not in stored])))
    return not_stored

In [99]:
informacion_personal = ["PERSONA. IDENTIFICACIÓN",
"NUMERO",
"NÚMERO DE PERSONA",
"HOGAR",
"AÑO DE LA ENCUESTA",
"MES DE LA ENCUESTA",
"DEPARTAMENTO",
"NOMBRE DEL DEPARTAMENTO",
"SECCIÓN CENSAL",
"SEGMENTO CENSAL",
"LOCALIDAD AGRUPADA",
"NOMBRE DE LA LOCALIDAD AGRUPADA",
"BARRIO",
"NOMBRE DEL BARRIO",
"ESTRATO",
"REGION_3",
"REGION_4"]
d_hogar = ['CENTRO COMUNAL ZONAL']
c_hogar = ['TENENCIA DE LA VIVIENDA']

ingresos_hogar = ['RECIBE DINERO DE ALGÚN FAMILIAR U OTRO HOGAR DEL PAÍS',
                'RECIBE AYUDA EN ESPECIE DE ALGÚN FAMILIAR U OTRO HOGAR DEL PAÍS',
                'HOGAR ALQUILA PARTE DE LA VIVIENDA QUE HABITA',
                'PROPIETARIO DE OTRA VIVIENDA O LOCAL',
                'PROPIETARIO DE CHACRA O CAMPO EN EL QUE NO TRABAJA',
                'INGRESO POR MEDIANERÍA SIN HABER TRABAJADO',
                'INGRESO POR PASTOREO SIN HABER TRABAJADO',
                'PROPIETARIO DE GANADO A CAPITALIZACIÓN',
                'INGRESO POR GANADO A CAPITALIZACIÓN',
                'RECIBIÓ INGRESOS POR ALQUILER DE VEHÍCULOS / MAQUINARIA',
                'LETRAS O BONOS',
                'PRÉSTAMOS A TERCEROS',
                'OTROS TÍTULOS',
                'PROPIETARIO DE NEGOCIO EN EL QUE NO TRABAJA',
                'RECIBIÓ INGRESOS POR MARCAS, PATENTES, DERECHOS DE AUTOR',
                'RECIBIÓ INDEMNIZACIÓN POR DESPIDO',
                'RECIBIÓ ALGUNA COLABORACIÓN ECONÓMICA DE ALGÚN FAMILIAR DEL EXTERIOR',
                'RECIBIÓ ALGÚN INGRESO EXTRAORDINARIO',
                 'MONTO QUE RECIBIÓ EL MES PASADO DE FAMILIAR EN EL PAIS',
                 'MONTO ESTIMADO DE LA AYUDA EN ESPECIEQUE RECIBIÓ EL MES PASADO',
                 'MONTO RECIBIDO POR ALQUILER DE PARTE DE LA VIVIENDA',
                 'FUERON ALQUILADAS SUS PROPIEDADES',
                 'MONTO RECIBIDO POR ALQUILER INTERIOR',
                 'MONTO RECIBIDO POR ALQUILER INTERIOR',
                 'MONTO RECIBIDO POR ALQUILER FUERA DEL PAIS',
                 'FUERON ARRENDADOS SU CHACRA QUE NO TRABAJA',
                 'MONTO PERCIBIDO POR ARRENDAMIENTOS DE CHACRA EN EL PAÍS',
                 'MONTO PERCIBIDO POR ARRENDAMIENTOS DE CHACRA DEL EXTRANJERO',
                 'MONTO RECIBIDO POR ALQUILER DE VEHICULOS',
                 'MONTO PERCIBIDO POR INTERESES BANCARIOS DEL PAÍS',
                 'MONTO PERCIBIDO POR INTERESES BANCARIOS DEL EXTRANJERO',
                 'MONTO PERCIBIDO POR INTERESES DE LETRAS DEL PAÍS',
                 'MONTO PERCIBIDO POR INTERESES DE LETRAS DEL EXTRANJERO',
                 'MONTO PERCIBIDO POR INTERESES DEL PAÍS',
                 'MONTO PERCIBIDO POR INTERESES DE PRESTAMOS A  TERCEROS DEL EXTRANJERO',
                 'MONTO PERCIBIDO POR OTROS TITULO SEN EL PAÍS',
                 'MONTO PERCIBIDO POR INTERESES POR OTROS TITULOS DEL EXTRANJERO',
                 'MONTO  QUE PERCIBE POR INGRESO DE NEGOCIO EN QUE NO TRABAJA EN URUGUAY',
                 'MONTO  QUE PERCIBE POR INGRESO DE NEGOCIO EN QUE NO TRABAJA EN EL EXTRANJERO',
                 'INGRESOS POR PATENTES O DERECHOS DE AUTOR',
                 'MONTO RECIBIDO POR DESPIDO',
                 'MONTO DE COLABORACIÓN ECONÓMICA DE ALGÚN FAMILIAR DEL EXTERIOR',
                 'MONTO DEL INGRESO EXTRAORDINARIO RECIBIDO',
                 'CONTRIBUCIONES POR DIVORCIO O SEPARACIÓN',
                 'MONTOS POR CONTRIBUCIONES POR DIVORCIO O SEPARACION',
                 'CONTRIBUCIONES EN DINERO O EN ESPECIE A OTROS HOGARES',
                 'MONTO DE CONTRIBUCIONES EN DINERO O EN ESPECIE A OTROS HOGARES']


variables_ag = ['CANTIDAD DE PERSONAS SIN SERVICIO DOMÉSTICO',
                 'INGRESOS DEL HOGAR NO IMPUTABLES A PERSONAS',
                 'INGRESO TOTAL DEL HOGAR SIN VALOR LOCATIVO SIN SERVICIO DOMÉSTICO',
                 'POBPCOAC',
                 'SUBEMPLEO',
                 'TIPO DE OCUPADO',
               'TOTAL DE INGRESOS PERSONALES',
                 'INGRESOS DE LA OCUPACIÓN PRINCIPAL',
                 'TOTAL DE INGRESOS POR TRABAJO',
               'POBREZA SEGÚN METODOLOGÍA 2006',
             'INDIGENCIA SEGÚN METODOLOGÍA 2006']


caracteristica_persona =   ['NUEVO MIEMBRO DEL HOGAR',
                            'NUEVO MIEMBRO DEL HOGAR DEPENDE DEL MISMO FONDO DE ALIMENTACIÓN',
                            'PERSONA CONTINUA RESIDIENDO EN EL HOGAR',
                            'IDENTIFICACIÓN DEL INFORMANTE',
                            'SEXO',
                            'EDAD',
                            'ASCENDENCIA',
                            'RELACIÓN DE PARENTESCO',
                           'AFRO O NEGRA',
                             'ASIÁTICA O AMARILLA',
                             'BLANCA',
                           'NUMERO DE PERSONA',
                         'INDÍGENA',
                         'OTRA',
                         'PRINCIPAL ASCENDENCIA']
general = ["MES","ANIO"]

educacion = ['ASISTENCIA A UN ESTABLECIMIENTO DE ENSEÑANZA',
             'TIPO DE CENTRO EDUCATIVO',
             'RECIBE ALIMENTACIÓN EN EL CENTRO EDUCATIVO',
             'DESAYUNOS Y/O MERIENDAS EN CENTRO EDUCATIVO',
             'ALMUERZOS / CENAS EN CENTRO EDUCATIVO',
            'MERIENDAS EN CENTRO EDUCATIVO',
             'SABE LEER Y ESCRIBIR',
             'ASISTENCIA ALGUNA VEZ A UN ESTABLECIMIENTO DE ENSEÑANZA',
             'ASISTENCIA A EDUCACIÓN INICIAL',
             'CANTIDAD DE DÍAS QUE ASISTE A LA SEMANA',
            'CANTIDAD DE HORAS AL DÍA',
             'FALTÓ ALGÚN DÍA',
             'RAZONES POR LAS QUE NO ASISTIÓ',
             'ASISTENCIA A EDUCACIÓN PREESCOLAR',
             'CANTIDAD DE HORAS AL DÍA',
            'DESAYUNOS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
             'ALMUERZOS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
             'CENAS SEMANALES EN PRIMARIA O CAIF PREESCOLAR',
             'ASISTENCIA A EDUCACIÓN PRIMARIA',
             'FINALIZÓ EL NIVEL',
             'AÑOS APROBADOS EN PRIMARIA COMÚN',
             'AÑOS APROBADOS EN PRIMARIA ESPECIAL',
             'DESAYUNOS SEMANALES',
             'ALMUERZOS SEMANALES',
             'CENAS SEMANALES',
            'ASISTENCIA A EDUCACIÓN MEDIA',
             'ABANDONO EDUCACIÓN POR COMENZÓ A TRABAJAR',
             'ABANDONO EDUCACIÓN POR NO TENÍA INTERÉS',
             'ABANDONO EDUCACIÓN POR LE INTERESABA APRENDER OTRAS COSAS',
             'ABANDONO EDUCACIÓN POR QUEDÓ USTED O SU PAREJA EMBARAZADA',
            'ABANDONO EDUCACIÓN POR LE RESULTABAN DIFÍCILES LA/S MATERIA/S',
             'ABANDONO EDUCACIÓN POR POR DIFICULTADES ECONÓMICAS (COMPRA DE MAT. DE ESTUDIO, ETC.)',
             'ABANDONO EDUCACIÓN POR PORQUE TUVO QUE ATENDER ASUNTOS FAMILIARES',
             'ABANDONO EDUCACIÓN POR OTRAS RAZONES',
             'ABANDONO EDUCACIÓN POR DESCRIPCIÓN OTRA RAZÓN DE NO FINALIZACIÓN',
            'PRINCIPAL RAZÓN DE NO FINALIZACIÓN',
             'AÑOS APROBADOS EN CICLO BÁSICO',
             'AÑOS APROBADOS EN BACHILLERATO',
             'AÑOS APROBADOS EN BACHILLERATO TECNOLÓGICO',
             'CURSO O CARRERA QUE ESTUDIA / ESTUDIÓ',
            'CANTIDAD DE COMIDAS SEMANALES',
             'CANTIDAD ALMUERZOS SEMANALES (EST. BACHILLERATO)',
             'MERIENDAS RECIBIDAS EN BACHILLERATO',
             'BECAS',
             'ASISTENCIA A MAGISTERIO O PROFESORADO',
             'AÑOS APROBADOS EN MAGISTERIO O PROFESORADO',
             'ASISTENCIA A UNIVERSIDAD O SIMILAR',
             'AÑOS APROBADOS EN UNIVERSIDAD O SIMILAR',
             'ASISTENCIA A TERCIARIO NO UNIVERSITARIO',
             'AÑOS APROBADOS EN TERCIARIO NO UNIVERSITARIO',
             'ASISTENCIA A POSGRADO',
             'AÑOS APROBADOS EN POSGRADO',
             'ASISTENCIA A EDUCACIÓN TÉCNICA',
             'AÑOS APROBADOS EN EDUCACIÓN TÉCNICA',
             'EXIGENCIA PARA REALIZAR CURSO DE EDUCACIÓN TÉCNICA'
            ]

salud = ['INSTITUCIÓN EN DONDE CUENTA CON DERECHOS PARA ATENDER SU SALUD',
        'MECANISMO ACCESO MSP/ASSE',
        'MECANISMO ACCESO IAMC',
        'MECANISMO ACCESO A SEGURO PRIVADO MÉDICO',
        'MECANISMO ACCESO A HOSPITAL POLICIAL/MILITAR',
        'AFILIACIÓN A EMERGENCIA MÓVIL',
     'QUIÉN PAGA LA CUOTA DE LA EMERGENCIA MÓVIL',
        ]


politicas_alimentarias = ['CONCURRENCIA A COMEDOR',
                         'DESAYUNOS Y/O MERIENDAS RECIBIDAS',
                         'ALMUERZOS Y/O CENAS RECIBIDAS',
                         'RECIBE ALGÚN TIPO DE TARJETA',
                         'TUS-MIDES',
                         'CANTIDAD RECIBIDA EL MES ANTERIOR (TUS-MIDES)',
                         'TUS-INDA',
                         'CANTIDAD RECIBIDA EL MES ANTERIOR (TUS-INDA)',
                         'LECHE EN POLVO',
                         'CANTIDAD DE LECHE EN POLVO',
                         'RECIBE ALGÚN TIPO DE CANASTA',
                         'TIPO DE CANASTA',
                         'CANTIDAD DE CANASTAS MENSUALES']

actividad_laboral = ['TRABAJÓ LA SEMANA PASADA',
                    'COLABORÓ EN NEGOCIO FAMILIAR O TRABAJO CONCRETO LA SEMANA PASADA',
                    'RECIBIÓ SUELDO',
                     'TRABAJO REALIZADO PARA UN NEGOCIO DEL HOGAR',
                     'NEGOCIO O ACTIVIDAD POR LA QUE PERCIBE UN PAGO',
                     'MOTIVO POR EL CUAL NO TRABAJÓ',
                     'FUE DESPEDIDO DE SU PUESTO DE TRABAJO',
                    'TIEMPO EN EL QUE VOLVERÁ A TRABAJAR',
                     'SI TERMINARAN RESTRICCIONES POR PANDEMIA, CREE VOLVER A SU PUESTO DE TRABAJO',
                     'REALIZÓ TAREAS AGRÍCOLAS PARA CONSUMO DEL HOGAR',
                     'DE LO QUE PRODUCE PARTE SE DESTINA A LA VENTA',
                     'TRABAJOS QUE TIENE',
                    'FUE DESPEDIDO DE SU PUESTO DE TRABAJO',
                     'TIEMPO EN EL QUE VOLVERÁ A TRABAJAR',
                     'TAREAS QUE PROPORCIONAN MAYORES INGRESOS',
                     'ACTIVIDAD DEL ESTABLECIMIENTO DONDE REALIZA LAS TAREAS',
                     'CATEGORÍA DE LA OCUPACIÓN',
                    'SECTOR PÚBLICO',
 'ESTABLECIMIENTO DONDE REALIZA LAS TAREAS PERTENECE A LA EMPRESA QUE LO CONTRATA',
 'A QUÉ SE DEDICA LA EMPRESA QUE LO CONTRATA',
 'NEGOCIO CON CONTABILIDAD COMPLETA ',
 'NEGOCIO REGISTRADO ANTE LAS OFICINAS DE IMPUESTOS O DE LA SEGURIDAD SOCIAL',
                    'NATURALEZA JURÍDICA DEL NEGOCIO',
 'FORMA DE TRIBUTACIÓN',
 'TAMAÑO DE LA EMPRESA ',
 'DÓNDE REALIZA SU TRABAJO',
 'TRABAJO EN EL DEPARTAMENTO',
                    'COBRA AGUINALDO',
 'APORTE A CAJA DE JUBILACIONES',
 'CAJA A LA CUAL APORTA',
 'APORTE POR TOTALIDAD DEL SALARIO',
 'RECONOCIMIENTO DE HORAS EXTRAS MEDIANTE PAGO, COMPENSACIÓN, DÍAS ADICIONALES DE VACACIONES',
                    'COMO PAGAN LAS HORAS EXTRAS',
 'HORARIO FLEXIBLE',
 'VACACIONES ANUALES PAGAS O PAGO LICENCIA NO GOZADA',
 'LICENCIA POR ENFERMEDAD O LESIÓN PAGA',
 'HORAS TRABAJADAS POR SEMANA',
                    'HORAS EFECTIVAS DE TRABAJO',
 'TIENE QUE LLEVAR TRABAJO PARA SU HOGAR (SOLO PARA ASALARIADOS)',
 'CANTIDAD DE HORAS SEMANALES DEDICADAS A ESE TRABAJO EN EL HOGAR (SOLO PARA ASALARIADOS)',
 'AÑOS QUE HACE DESDE QUE TRABAJA',
 'ESTUVO DESOCUPADO EN LOS ÚLTIMOS 12 MESES',
                    'TAREAS QUE REALIZA EN EL OTRO TRABAJO',
 'A QUÉ SE DEDICA EL ESTABLECIMIENTO DONDE REALIZA LAS TAREAS',
 'APORTE CAJA JUBILACIONES',
 'HORAS TRABAJADAS HABITUALMENTE EN OTRAS OCUPACIONES',
 'HORAS EFECTIVAS TRABAJADAS EN OTRAS OCUPACIONES',
                    'TELETRABAJO HABITUAL',
 'REALIZÓ TELETRABAJO',
 'BUSCA OTRO TRABAJO',
 'SUSTITUIR O COMPLEMENTAR EL ACTUAL',
 'RAZONES POR LAS CUALES QUIERE OTRO EMPLEO',
                    'DESEA TRABAJAR MÁS HORAS',
 'ESTÁ DISPONIBLE PARA TRABAJAR MÁS HORAS',
 'RAZONES POR LAS CUALES NO TRABAJA MÁS HORAS A LA SEMANA',
 'QUÉ HIZO PARA TRABAJAR MÁS HORAS',
 'DISPONIBILIDAD PARA TRABAJAR',
                    'LA SEMANA PASADA BUSCÓ TRABAJO',
 'RAZONES POR LAS CUALES NO BUSCÓ TRABAJO NI ESTABLECIÓ SU NEGOCIO',
 'BUSCÓ TRABAJO EN LAS ÚLTIMAS CUATRO SEMANAS',
 'QUÉ HIZO PARA BUSCAR TRABAJO O ESTABLECER SU PROPIO NEGOCIO',
 'BUSCA TRABAJO CON CONDICIONES ESPECIALES',
                    'PRINCIPAL CONDICIÓN QUE REQUIERE',
 'TIEMPO QUE BUSCA TRABAJO',
 'REALIZÓ GESTIONES CONCRETAS',
 'REALIZÓ ALGÚN TRABAJO OCASIONAL',
 'HA TRABAJADO ANTES',
                    'ESTÁ EN SEGURO DE DESEMPLEO',
 'TIEMPO QUE HACE QUE DEJÓ EL ÚLTIMO EMPLEO (MESES)',
 'TAREAS QUE REALIZABA EN ESA OCUPACIÓN',
 'A QUÉ SE DEDICABA O QUÉ PRODUCÍA EL ESTABLECIMIENTO EN QUE REALIZABA SUS TAREAS',
 'RAZONES POR LAS CUALES DEJÓ EL TRABAJO',
                    'ESTÁ EN SEGURO DE DESEMPLEO',
 'TIEMPO QUE HACE QUE DEJÓ EL ÚLTIMO EMPLEO (MESES)',
 'TAREAS QUE REALIZABA EN ESA OCUPACIÓN',
 'A QUÉ SE DEDICABA O QUÉ PRODUCÍA EL ESTABLECIMIENTO EN QUE REALIZABA SUS TAREAS',
 'RAZONES POR LAS CUALES DEJÓ EL TRABAJO',
                    'APORTABA A CAJA DE JUBILACIONES',
 'JUBILADO/A',
 'PENSIONISTA',
 'RENTISTA',
 'ESTUDIANTE',
                    'QUIEN REALIZA PRINCIPALMENTE LOS QUEHACERES DEL HOGAR',
 'TIPO DE PENSIÓN QUE RECIBE',
 'SUELDO O JORNALES LÍQUIDOS',
 'COMISIONES, INCENTIVOS, HORAS EXTRAS, HABILITACIONES',
 'VIÁTICOS NO SUJETOS A RENDICIÓN',
                    'PROPINAS',
 'AGUINALDO',
 'SALARIO VACACIONAL',
 'PAGOS ATRASADOS',
 'BOLETOS DE TRANSPORTE',
 'COBRA EN EFECTIVO / CHEQUE',
 'COBRA POR DEPÓSITO EN CUENTA BANCARIA',
 'COBRA TARJETA PREPAGA (DEANDA, MIDINERO, ETC.)',
                    'COBRA POR GIROS O TRANSFERENCIAS BANCARIAS',
 'COBRA POR GIROS O TRANSFERENCIAS NO BANCARIAS',
 'RECIBIÓ ALIMENTOS O BEBIDAS',
 'RECIBIO  DESAYUNOS / MERIENDAS',
 'RECIBIO ALMUERZOS / CENAS',
 ' MONTO ESTIMADO RECIBIDO DE ALIMENTOS',
 'RECIBIÓ TIQUES DE ALIMENTACIÓN',
 'VALOR RECIBIDO EL MES PASADO',
                    'RECIBIÓ VIVIENDA O ALOJAMIENTO',
 'ESTÁ EN EL MEDIO URBANO',
 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESE ALOJAMIENTO',
 'RECIBIÓ OTRO TIPO DE RETRIBUCIÓN EN ESPECIE',
 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS BIENES',
 'RECIBIÓ ALGÚN OTRO TIPO DE COMPLEMENTO PAGADO POR EL EMPLEADOR',
 'MONTO PERCIBIDO COMPLEMENTO PAGADOR',
 'DERECHO A PASTOREO TRABAJO PRINCIPAL',
 'CANTIDAD DE VACUNOS',
 'CANTIDAD DE OVINOS',
 'CANTIDAD DE EQUINOS',
 'DERECHO A CULTIVAR PARA CONSUMO PROPIO',
                    'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS PRODUCTOS',
 'MONTO PERCIBIDO POR LA VENTA DE ESOS PRODUCTOS',
 'INGRESOS DEL MES ANTERIOR POR ALGÚN OTRO EMPLEO ASALARIADO',
 'SUELDOS O JORNALES LÍQUIDOS',
 'EN EFECTIVO / CHEQUE',
 'POR DEPÓSITO EN CUENTA BANCARIA',
 'TARJETA PREPAGA (DEANDA, MIDINERO, ETC.)',
 'POR GIROS O TRANSFERENCIAS BANCARIAS',
 'POR GIROS O TRANSFERENCIAS NO BANCARIAS',
 'CANTIDAD DE DESAYUNOS / MERIENDAS RECIBIDAS',
 'CANTIDAD DE ALMUERZOS / CENAS RECIBIDAS',
 'OTROS - MONTO ESTIMADO RECIBIDAS',
                    'MONTO TICKET ALIMENTACION',
 'MEDIO URBANO',
 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR ESOS BIENES DE OTROS BIENES',
 'RECIBIÓ ALGÚN OTRO COMPLEMENTO PAGADO POR EL EMPLEADOR',
 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR  EL COMPLEMENTO EMPLEADOR',
 'DERECHO A PASTOREO',
 'VACUNOS PASTOREANDO',
 'OVINOS PASTOREANDO',
 'EQUINOS PASTOREANDO',
 'DERECHO A CULTIVAR ',
 'MONTO QUE HABRÍA TENIDO QUE PAGAR POR CULTIVOS SI LOS TUVIERA QUE COMPRAR',
 'MONTO PERCIBIDO POR LA VENTA DE ESOS PRODUCTOS QUE CULTIVO',
                    'RETIRÓ DINERO PARA GASTOS DEL HOGAR DE NEGOCIOS QUE TIENE O TENÍA',
 'MONTO DE DINERO QUE RETIRÓ  PARA GASTOS DEL HOGAR DE NEGOCIOS QUE TIENE O TENÍA',
 'INGRESOS POR DISTRIBUCIÓN DE UTILIDADES POR NEGOCIO QUE TIENE O TENÍA',
 'INGRESOS POR DISTRIBUCIÓN DE UTILIDADES POR NEGOCIO QUE TIENE O TENÍA',
 'RETIRO DE PRODUCTOS PARA CONSUMO PROPIO (TRABAJADOR NO AGROPECUARIO)',
 'MONTO DE RETIRO DE PRODUCTOS PARA CONSUMO PROPIO (TRABAJADOR NO AGROPECUARIO)',
 'VALOR DE LO CONSUMIDO EN CARNES O CHACINADOS',
 'VALOR DE LO CONSUMIDO EN LÁCTEOS',
 'VALOR DE LO CONSUMIDO EN HUEVOS Y AVES',
 'VALOR DE LO CONSUMIDO EN PRODUCTOS DE LA HUERTA',
 'VALOR CONSUMIDO EN OTROS ALIMENTOS',
 'RECIBIÓ INGRESOS POR MEDIANERÍA O APARCERÍA',
                    'INGRESO POR MEDIANERÍA O APARCERÍA',
 'INGRESO POR PASTOREO']

jubilaciones = ['RECIBIÓ INGRESOS POR JUBILACIONES',
                 'MONTO APORTES DE BPS - CAJA INDUSTRIA Y COMERCIO',
                 'MONTO APORTES DEBPS - CAJA CIVIL Y ESCOLAR',
                 'MONTO APORTES BPS - RURAL Y SERVICIO DOMÉSTICO',
                 'MONTO APORTES UNIÓN POSTAL',
                 'MONTO APORTES POLICIAL',
                 'MONTO APORTES MILITAR',
                 'MONTO APORTES CAJA PROFESIONAL',
                 'MONTO APORTES CAJA NOTARIAL',
                 'MONTO APORTES CAJA BANCARIA',
                'MONTO APORTES AFAP (INGRESOS PROVENIENTES ÚNICAMENTE DE ALGUNA AFAP)',
                 'MONTO APORTES OTRA CAJA',
                 'MONTO APORTES OTRO PAÍS']
pensiones = [ 'RECIBIÓ INGRESOS POR PENSIONES',
             'BPS - CAJA INDUSTRIA Y COMERCIO',
             'BPS - CAJA CIVIL Y ESCOLAR',
             'BPS - RURAL Y SERVICIO DOMÉSTICO',
             'UNIÓN POSTAL',
             'POLICIAL',
             'MILITAR',
             'PROFESIONAL',
             'NOTARIAL',
            'BANCARIA',
             'AFAP (INGRESOS PROVENIENTES ÚNICAMENTE DE ALGUNA AFAP)',
             'OTRO PAÍS',
             'SEGURO DE DESEMPLEO',
             'COMPENSACIONES POR ACCIDENTE, MATERNIDAD O ENFERMEDAD',
             'BECAS, SUBSIDIOS, DONACIONES',
             'BECAS, SUBSIDIOS, DONACIONES DESDE EL EXTRANJERO',
             'COBRA HOGAR CONSTITUIDO',
             'COBRA HOGAR CONSTITUIDO DECLARADO EN EL SUELDO',
             'COBRA ASIGNACIONES FAMILIARES',
             'RECIBE LAS ASIGNACIONES POR EL PLAN DE EQUIDAD DEL MIDES',
             'FORMA DE COBRO DE LAS ASIGNACIONES FAMILIARES',
            'FRECUENCIA DE COBRO DE ASIGNACIONES FAMILIARES',
             'CANTIDAD DE ASIGNACIONES FAMILIARES',
             'CANTIDAD DE ASIGNACIONES DE MENORES QUE ESTAN EN SECUNDARIA',
             'CANTIDAD DE ASIGNACIONES A PERSONAS CON DISCAPACIDAD',
             'MONTO PERCIBIDO POR ASIGNACIONES FAMILIARES',
             'RECIBE PENSIÓN ALIMENTICIA O ALGUNA CONTRIBUCIÓN POR DIVORCIO O SEPARACIÓN',
             'RECIBE PENCION ALIMENTICIA POR DIVORCIO EN EL PAÍS',
             'RECIBE PENCION ALIMENTICIA POR DIVORCIO EN EL EXTERIOR',
              'DEVOLUCIÓN POR FONASA',
             'OTRO INGRESO CORRIENTE ADEMÁS DE LOS DECLARADOS',
            'MONTO DEVOLUCION FONASA']

In [100]:
### Columnas separadas por categorrias. Servira para subsegmentar el dataset
### PUede hacerse programaticamente la categorizacion pero es necesario un control de errores
dict_columns = {"General": general, 
                "Variables agregadas en Hogar":variables_ag,
                "Información personal": informacion_personal,
               'D.Hogar' : d_hogar,
               'C.Hogar': c_hogar,
               'Ingresos hogar':ingresos_hogar,
               'Caracteristica de la persona' : caracteristica_persona,
               'Salud':salud,
               'Educacion':educacion,
               'Politicas Alimentarias':politicas_alimentarias,
               'Actividad Laboral':actividad_laboral,
               "Jubilaciones":jubilaciones,
               "Pensiones":pensiones}

In [101]:
def select_categories_and_save_csv(df,dict_columns,list_categories=[]):
    result_columns = []
    if len(list_categories) == 0:
        df.to_csv('Encuesta de hogares procesada Completa.csv',index=False,encoding='latin-1',errors='ignore')
        print("Archivo total guardado con exito")
    else:
        filename = 'Encuesta de hogares procesada con variables ' + '_'.join(list_categories) + '.csv'
        
        for category in list_categories:
            if category not in dict_columns.keys():
                print("{} no esta en lista de categorias".format(category))
                print("Debes ingresar alguna de las siguientes:")
                print([x for x in dict_columns.keys()])
                break
            else:
                columns = dict_columns.get(category)
                result_columns += columns
        result_df = df.copy()
        result_df = result_df[columns]
        df.to_csv(filename,index=False,encoding='latin-1',errors='ignore')
        print("{} guardado con exito".format(filename))

In [102]:
select_categories_and_save_csv(df,dict_columns)
select_categories_and_save_csv(df,dict_columns,['General',"Información personal",'Caracteristica de la persona','Salud'])
select_categories_and_save_csv(df,dict_columns,['General',"Información personal",'Caracteristica de la persona','Actividad Laboral','Ingresos hogar'])
select_categories_and_save_csv(df,dict_columns,['General',"Información personal",'Caracteristica de la persona','D.Hogar','C.Hogar'])
select_categories_and_save_csv(df,dict_columns,['General',"Información personal",'Caracteristica de la persona','Actividad Laboral','Ingresos hogar'])

Archivo total guardado con exito
Encuesta de hogares procesada con variables General_Información personal_Caracteristica de la persona_Salud.csv guardado con exito
Encuesta de hogares procesada con variables General_Información personal_Caracteristica de la persona_Actividad Laboral_Ingresos hogar.csv guardado con exito
Encuesta de hogares procesada con variables General_Información personal_Caracteristica de la persona_D.Hogar_C.Hogar.csv guardado con exito
Encuesta de hogares procesada con variables General_Información personal_Caracteristica de la persona_Actividad Laboral_Ingresos hogar.csv guardado con exito
